Welcome to this example script of building electrification HVAC automation modeling. We will explain step by step what this script does using an example with two input files, one is the target file to be electrified, and the other one is the source file of electrified HVAC reference.

First step, install EPPY.

In [1]:
pip install eppy

Note: you may need to restart the kernel to use updated packages.


Install all packages that might be used

In [2]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import pandas as pd
import os.path
import glob

Locate IDD file. In this case I saved IDD under the same folder with this notebook.

In [3]:
iddfile = 'Energy+.idd'
IDF.setiddname(iddfile)

Import two idf files. The target idf is the model to be electrified. The source idf is the model with the electrified HVAC system, regardless whether it's the same building type as the target.

In [4]:
#target file is DOE reference building of small office
targetfname = 'RefBldgSmallOfficePost1980_v1.4_7.2_2A_USA_TX_HOUSTON.idf'
targetidf = IDF(targetfname)

In [5]:
#source file is a SFH with ASHP
sourcefname = 'US+SF+CZ4A+ASHP+ElectricStrip+slab+IECC_2006_EnergyStar v6.idf'
sourceidf = IDF(sourcefname)

# Find target zones

We first identify the conditioned zones in the target model. This involve one manual step to rule out the unconditioned zone. The conditioned zones will be used in the later step of HVAC electrification replacement.

In [6]:
#Number of zones in the target model
print(len(targetidf.idfobjects['Zone']))

6


In [7]:
#Name of zones, both conditioned and unconditioned, in the target model
zones=[]
for i in range(len(targetidf.idfobjects['Zone'])):
    zones.append(targetidf.idfobjects['Zone'][i].Name)

zones

['Attic',
 'Core_ZN',
 'Perimeter_ZN_1',
 'Perimeter_ZN_2',
 'Perimeter_ZN_3',
 'Perimeter_ZN_4']

In [8]:
#Rule out unconditioned zone(s) - this step needs manual observation.
zones.remove('Attic')
zones

['Core_ZN',
 'Perimeter_ZN_1',
 'Perimeter_ZN_2',
 'Perimeter_ZN_3',
 'Perimeter_ZN_4']

# Find Unnecessary HVAC Objects in the Target File and Clear Them for Replacement

In [9]:
#review target file object types
target_obj=[]
for obj_type in targetidf.idfobjects:
    for obj in targetidf.idfobjects[obj_type]:
        target_obj.append(obj_type)

unique_target_list=[]        
[unique_target_list.append(x) for x in target_obj if x not in unique_target_list]
unique_target_list


['VERSION',
 'SIMULATIONCONTROL',
 'BUILDING',
 'SHADOWCALCULATION',
 'SURFACECONVECTIONALGORITHM:INSIDE',
 'SURFACECONVECTIONALGORITHM:OUTSIDE',
 'HEATBALANCEALGORITHM',
 'ZONEAIRHEATBALANCEALGORITHM',
 'TIMESTEP',
 'CONVERGENCELIMITS',
 'SITE:LOCATION',
 'SIZINGPERIOD:DESIGNDAY',
 'RUNPERIOD',
 'RUNPERIODCONTROL:SPECIALDAYS',
 'RUNPERIODCONTROL:DAYLIGHTSAVINGTIME',
 'SITE:GROUNDTEMPERATURE:BUILDINGSURFACE',
 'SITE:WATERMAINSTEMPERATURE',
 'SCHEDULETYPELIMITS',
 'SCHEDULE:COMPACT',
 'MATERIAL',
 'MATERIAL:NOMASS',
 'WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM',
 'CONSTRUCTION',
 'GLOBALGEOMETRYRULES',
 'ZONE',
 'BUILDINGSURFACE:DETAILED',
 'FENESTRATIONSURFACE:DETAILED',
 'INTERNALMASS',
 'PEOPLE',
 'LIGHTS',
 'ELECTRICEQUIPMENT',
 'ZONEINFILTRATION:DESIGNFLOWRATE',
 'EXTERIOR:LIGHTS',
 'DESIGNSPECIFICATION:OUTDOORAIR',
 'SIZING:PARAMETERS',
 'SIZING:ZONE',
 'SIZING:SYSTEM',
 'SIZING:PLANT',
 'ZONECONTROL:THERMOSTAT',
 'THERMOSTATSETPOINT:DUALSETPOINT',
 'AIRTERMINAL:SINGLEDUCT:CONSTANTVOLUME:

In [10]:
#Select HVAC related object types - This step is manual
target_HVAC_list=['SIZING:SYSTEM','ZONEHVAC:AIRDISTRIBUTIONUNIT','ZONEHVAC:EQUIPMENTLIST','ZONEHVAC:EQUIPMENTCONNECTIONS',
                  'FAN:CONSTANTVOLUME','COIL:COOLING:DX:SINGLESPEED','COIL:HEATING:FUEL','COILSYSTEM:COOLING:DX',
                  'CONTROLLER:OUTDOORAIR', 'AIRLOOPHVAC:CONTROLLERLIST', 'AIRLOOPHVAC',
                  'AIRLOOPHVAC:OUTDOORAIRSYSTEM:EQUIPMENTLIST', 'AIRLOOPHVAC:OUTDOORAIRSYSTEM', 'OUTDOORAIR:MIXER', 'AIRLOOPHVAC:ZONESPLITTER',
                  'AIRLOOPHVAC:SUPPLYPATH', 'AIRLOOPHVAC:ZONEMIXER', 'AIRLOOPHVAC:RETURNPATH', 'BRANCH', 'BRANCHLIST',
                  'CONNECTOR:SPLITTER', 'CONNECTOR:MIXER', 'CONNECTORLIST', 'NODELIST', 'OUTDOORAIR:NODE', 'OUTDOORAIR:NODELIST',
                  'AVAILABILITYMANAGER:NIGHTCYCLE', 'AVAILABILITYMANAGERASSIGNMENTLIST', 'SETPOINTMANAGER:SCHEDULED',
                  'SETPOINTMANAGER:SINGLEZONE:REHEAT', 'SETPOINTMANAGER:MIXEDAIR','CURVE:QUADRATIC', 'CURVE:CUBIC', 'CURVE:BIQUADRATIC']

In [11]:
#review source file objects
source_obj=[]
for obj_type in sourceidf.idfobjects:
    for obj in sourceidf.idfobjects[obj_type]:
        source_obj.append(obj_type)

unique_source_list=[]        
[unique_source_list.append(x) for x in source_obj if x not in unique_source_list]
unique_source_list


['VERSION',
 'SIMULATIONCONTROL',
 'BUILDING',
 'SURFACECONVECTIONALGORITHM:INSIDE',
 'TIMESTEP',
 'SITE:LOCATION',
 'SIZINGPERIOD:DESIGNDAY',
 'RUNPERIOD',
 'SITE:WATERMAINSTEMPERATURE',
 'SCHEDULETYPELIMITS',
 'SCHEDULE:DAY:HOURLY',
 'SCHEDULE:WEEK:COMPACT',
 'SCHEDULE:YEAR',
 'SCHEDULE:COMPACT',
 'SCHEDULE:CONSTANT',
 'MATERIAL',
 'MATERIAL:NOMASS',
 'MATERIAL:AIRGAP',
 'WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM',
 'WINDOWMATERIAL:GLAZING',
 'WINDOWMATERIAL:BLIND',
 'CONSTRUCTION',
 'GLOBALGEOMETRYRULES',
 'ZONE',
 'BUILDINGSURFACE:DETAILED',
 'WINDOW',
 'DOOR',
 'WINDOWSHADINGCONTROL',
 'INTERNALMASS',
 'SHADING:OVERHANG',
 'GROUNDHEATTRANSFER:CONTROL',
 'GROUNDHEATTRANSFER:SLAB:MATERIALS',
 'GROUNDHEATTRANSFER:SLAB:MATLPROPS',
 'GROUNDHEATTRANSFER:SLAB:BOUNDCONDS',
 'GROUNDHEATTRANSFER:SLAB:BLDGPROPS',
 'GROUNDHEATTRANSFER:SLAB:INSULATION',
 'GROUNDHEATTRANSFER:SLAB:EQUIVALENTSLAB',
 'GROUNDHEATTRANSFER:BASEMENT:SIMPARAMETERS',
 'GROUNDHEATTRANSFER:BASEMENT:MATLPROPS',
 'GROUNDHEATTRANSF

In [12]:
#Select HVAC related object types - This step is manual
source_HVAC_list=['SIZING:SYSTEM','AIRTERMINAL:SINGLEDUCT:CONSTANTVOLUME:NOREHEAT', 'ZONEHVAC:AIRDISTRIBUTIONUNIT',
                  'ZONEHVAC:EQUIPMENTLIST', 'ZONEHVAC:EQUIPMENTCONNECTIONS', 'FAN:ONOFF', 'COIL:COOLING:DX:SINGLESPEED',
                  'COIL:HEATING:ELECTRIC', 'COIL:HEATING:DX:SINGLESPEED', 'AIRLOOPHVAC:UNITARYHEATPUMP:AIRTOAIR', 'AIRLOOPHVAC',
                  'AIRLOOPHVAC:ZONESPLITTER', 'AIRLOOPHVAC:SUPPLYPATH', 'AIRLOOPHVAC:ZONEMIXER', 'AIRLOOPHVAC:RETURNPATH',
                  'BRANCH', 'BRANCHLIST', 'CONNECTOR:SPLITTER', 'CONNECTOR:MIXER', 'CONNECTORLIST', 'NODELIST',
                  'AVAILABILITYMANAGER:SCHEDULED', 'AVAILABILITYMANAGERASSIGNMENTLIST', 'SETPOINTMANAGER:SCHEDULED',
                  'CURVE:QUADRATIC', 'CURVE:CUBIC', 'CURVE:BIQUADRATIC']

In [13]:
#This is the list of HVAC realted object types that exist in the target file but not the source file. These objects are no longer needed.
HVAC_dif_list_target=[obj for obj in target_HVAC_list if obj not in source_HVAC_list]
HVAC_dif_list_target

['FAN:CONSTANTVOLUME',
 'COIL:HEATING:FUEL',
 'COILSYSTEM:COOLING:DX',
 'CONTROLLER:OUTDOORAIR',
 'AIRLOOPHVAC:CONTROLLERLIST',
 'AIRLOOPHVAC:OUTDOORAIRSYSTEM:EQUIPMENTLIST',
 'AIRLOOPHVAC:OUTDOORAIRSYSTEM',
 'OUTDOORAIR:MIXER',
 'OUTDOORAIR:NODE',
 'OUTDOORAIR:NODELIST',
 'AVAILABILITYMANAGER:NIGHTCYCLE',
 'SETPOINTMANAGER:SINGLEZONE:REHEAT',
 'SETPOINTMANAGER:MIXEDAIR']

There are a few object types we would like to keep in order to make sure we are comparing apple to apple, for example, availability manager. We create a user defined exclusion keyword list and clear all other unnecessary objects in the target file.

In [14]:
def keep_objtype_without_keyword(lst, keywords):
    return [item for item in lst if not any(keyword in item for keyword in keywords)]

In [15]:
#Create an exception list, this is user defined. Make sure the keyword(s) is/are capitalized. 
keep_keys=['AVAILABILITY']

delete_lst=keep_objtype_without_keyword(HVAC_dif_list_target, keep_keys)
delete_lst

['FAN:CONSTANTVOLUME',
 'COIL:HEATING:FUEL',
 'COILSYSTEM:COOLING:DX',
 'CONTROLLER:OUTDOORAIR',
 'AIRLOOPHVAC:CONTROLLERLIST',
 'AIRLOOPHVAC:OUTDOORAIRSYSTEM:EQUIPMENTLIST',
 'AIRLOOPHVAC:OUTDOORAIRSYSTEM',
 'OUTDOORAIR:MIXER',
 'OUTDOORAIR:NODE',
 'OUTDOORAIR:NODELIST',
 'SETPOINTMANAGER:SINGLEZONE:REHEAT',
 'SETPOINTMANAGER:MIXEDAIR']

In [16]:
#Clear these unnecessary objects in the target file
for obj in delete_lst:
    targetidf.idfobjects[obj].clear()
    

Now we have cleared the unnecessary HVAC objects in the target file. Next, we need to edit the HVAC related objects one by one. HVAC related object types can be categorized into Sizing, AirTerminal, ZoneHVAC, Fan, Coil, Controller, OutdoorAir, AirLoopHVAC, Branch, BranchList, NodeList, AvailabilityManager, Curve and Schedule. We will go over each category and edit each object type one by one.

Overall, these categories can be further grouped into different groups:

1. Setting: Sizing

2. Mechanical Components: AirTeminal, ZoneHVAC, Fan, Coil, Controller, OutdoorAir, AirLoopHVAC

3. NonMechanical Components: Branch, BranchList, NodeList, AvailabilityManager

4. Mathematical Components: Curve, Schedule

For Setting, Mechanical and NonMechanical Components, the main logic follows theses steps:

1. Identify the object type(s) that belong to this object category in both target and source files;

2. Identify the existing object(s) of the specific object type in both files, read them carefully;

3. Clear the existing object(s) in the target file;

4. Duplicate ONE representative existing object from the source file to the target file;

5. Rename the duplicated object and the associated components/nodes by adopting the original name from the source file suffix with ONE conditione zone name in the target file;

6. Repeat the duplication in a loop over the conditioned zones in the target file.

We try to prioritze modifying the component objects first then the system objects, so that in some system objects (e.g. AirLoopHVAC), predefined components can be directly cited and connected to the system without repeating the above naming process to avoid potential errors.

For Mathematical Components, specific procedures are assigned.

See the below example. Each object type identified in the object category need to be edit one by one, as each object type requires different inputs.

# Edit Sizing

In [17]:
#Identify HVAC related SIZING object types exist in both files. 
#SIZING needs to be capitalized as object types are capitalized in idf files.
sizing_target=[s for s in target_HVAC_list if "SIZING" in s]
sizing_source=[s for s in source_HVAC_list if "SIZING" in s]

sizing_both=list(set(sizing_target+sizing_source))
sizing_both

['SIZING:SYSTEM']

In [18]:
#Find the Sizing:System in the target file.
#Sizing:System should not be all capitalized as in the idf files the name line of the specific object is not capitalized.
lst_sizingsystem = targetidf.idfobjects['Sizing:System']
lst_sizingsystem

[
Sizing:System,
    PSZ-AC:1,                 !- AirLoop Name
    Sensible,                 !- Type of Load to Size On
    AUTOSIZE,                 !- Design Outdoor Air Flow Rate
    1,                        !- Central Heating Maximum System Air Flow Ratio
    7,                        !- Preheat Design Temperature
    0.008,                    !- Preheat Design Humidity Ratio
    12.8,                     !- Precool Design Temperature
    0.008,                    !- Precool Design Humidity Ratio
    12.8,                     !- Central Cooling Design Supply Air Temperature
    40,                       !- Central Heating Design Supply Air Temperature
    NonCoincident,            !- Type of Zone Sum to Use
    No,                       !- 100 Outdoor Air in Cooling
    No,                       !- 100 Outdoor Air in Heating
    0.0085,                   !- Central Cooling Design Supply Air Humidity Ratio
    0.008,                    !- Central Heating Design Supply Air Humidity 

In [19]:
#Find the Sizing:System in the source file.
sourceidf.idfobjects['Sizing:System']

[
Sizing:System,
    Central System_unit1,     !- AirLoop Name
    Sensible,                 !- Type of Load to Size On
    autosize,                 !- Design Outdoor Air Flow Rate
    1,                        !- Central Heating Maximum System Air Flow Ratio
    7,                        !- Preheat Design Temperature
    0.008,                    !- Preheat Design Humidity Ratio
    11,                       !- Precool Design Temperature
    0.008,                    !- Precool Design Humidity Ratio
    12,                       !- Central Cooling Design Supply Air Temperature
    50,                       !- Central Heating Design Supply Air Temperature
    NonCoincident,            !- Type of Zone Sum to Use
    No,                       !- 100 Outdoor Air in Cooling
    No,                       !- 100 Outdoor Air in Heating
    0.008,                    !- Central Cooling Design Supply Air Humidity Ratio
    0.008,                    !- Central Heating Design Supply Air Humidity 

In [20]:
#Clear the orignial Sizing:System in the target
targetidf.idfobjects['Sizing:System'].clear()

#Add the Sizing:System from source to target. 
#Rename the object by original object name in source file + conditioned zone's name
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Sizing:System'][-1]
    targetidf.copyidfobject(newclg)
    lst_sizingsystem[-1].AirLoop_Name=newclg.AirLoop_Name+' '+zones[i]
    
lst_sizingsystem

[
Sizing:System,
    Central System_unit1 Core_ZN,    !- AirLoop Name
    Sensible,                 !- Type of Load to Size On
    autosize,                 !- Design Outdoor Air Flow Rate
    1,                        !- Central Heating Maximum System Air Flow Ratio
    7,                        !- Preheat Design Temperature
    0.008,                    !- Preheat Design Humidity Ratio
    11,                       !- Precool Design Temperature
    0.008,                    !- Precool Design Humidity Ratio
    12,                       !- Central Cooling Design Supply Air Temperature
    50,                       !- Central Heating Design Supply Air Temperature
    NonCoincident,            !- Type of Zone Sum to Use
    No,                       !- 100 Outdoor Air in Cooling
    No,                       !- 100 Outdoor Air in Heating
    0.008,                    !- Central Cooling Design Supply Air Humidity Ratio
    0.008,                    !- Central Heating Design Supply Air Hu

In [21]:
#Check if the new Sizing:System has the right count same as the conditioned zone count
len(lst_sizingsystem)

5

# Edit AirTerminal

In [22]:
#Identify HVAC related AIRTERMINAL object types exist in both files. 
#AIRTERMINAL needs to be capitalized as object types are capitalized in idf files.
at_target=[s for s in target_HVAC_list if "AIRTERMINAL" in s]
at_source=[s for s in source_HVAC_list if "AIRTERMINAL" in s]

at_both=list(set(at_target+at_source))
at_both

['AIRTERMINAL:SINGLEDUCT:CONSTANTVOLUME:NOREHEAT']

In [23]:
#Find the AirTerminal:SingleDuct:ConstantVolume:NoReheat in the target file.
#Sizing:System should not be all capitalized as in the idf files the name line of the specific object is not capitalized.
lst_CAV = targetidf.idfobjects['AirTerminal:SingleDuct:ConstantVolume:NoReheat']
lst_CAV

[
AirTerminal:SingleDuct:ConstantVolume:NoReheat,
    Core_ZN Direct Air,       !- Name
    ALWAYS_ON,                !- Availability Schedule Name
    Core_ZN Direct Air Inlet Node Name ATInlet,    !- Air Inlet Node Name
    Core_ZN Direct Air Inlet Node Name,    !- Air Outlet Node Name
    AUTOSIZE;                 !- Maximum Air Flow Rate
, 
AirTerminal:SingleDuct:ConstantVolume:NoReheat,
    Perimeter_ZN_1 Direct Air,    !- Name
    ALWAYS_ON,                !- Availability Schedule Name
    Perimeter_ZN_1 Direct Air Inlet Node Name ATInlet,    !- Air Inlet Node Name
    Perimeter_ZN_1 Direct Air Inlet Node Name,    !- Air Outlet Node Name
    AUTOSIZE;                 !- Maximum Air Flow Rate
, 
AirTerminal:SingleDuct:ConstantVolume:NoReheat,
    Perimeter_ZN_2 Direct Air,    !- Name
    ALWAYS_ON,                !- Availability Schedule Name
    Perimeter_ZN_2 Direct Air Inlet Node Name ATInlet,    !- Air Inlet Node Name
    Perimeter_ZN_2 Direct Air Inlet Node Name,    !- Air Ou

In [24]:
sourceidf.idfobjects['AirTerminal:SingleDuct:ConstantVolume:NoReheat']

[
AirTerminal:SingleDuct:ConstantVolume:NoReheat,
    ZoneDirectAir_unit1,      !- Name
    always_avail,             !- Availability Schedule Name
    Zone Inlet Node_unit1 ATInlet,    !- Air Inlet Node Name
    Zone Inlet Node_unit1,    !- Air Outlet Node Name
    autosize;                 !- Maximum Air Flow Rate
]

In [24]:
#Clear the orignial AirTerminal:SingleDuct:ConstantVolume:NoReheat in the target
targetidf.idfobjects['AirTerminal:SingleDuct:ConstantVolume:NoReheat'].clear()

#Add the AirTerminal:SingleDuct:ConstantVolume:NoReheat from source to target. 
#Rename the object by original object name in source file + conditioned zone's name
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['AirTerminal:SingleDuct:ConstantVolume:NoReheat'][-1]
    targetidf.copyidfobject(newclg)
    lst_CAV[-1].Name=newclg.Name+' '+zones[i]
    lst_CAV[-1].Air_Inlet_Node_Name=newclg.Air_Inlet_Node_Name+' '+zones[i]
    lst_CAV[-1].Air_Outlet_Node_Name=newclg.Air_Outlet_Node_Name+' '+zones[i]
    
lst_CAV

[
AirTerminal:SingleDuct:ConstantVolume:NoReheat,
    ZoneDirectAir_unit1 Core_ZN,    !- Name
    always_avail,             !- Availability Schedule Name
    Zone Inlet Node_unit1 ATInlet Core_ZN,    !- Air Inlet Node Name
    Zone Inlet Node_unit1 Core_ZN,    !- Air Outlet Node Name
    autosize;                 !- Maximum Air Flow Rate
, 
AirTerminal:SingleDuct:ConstantVolume:NoReheat,
    ZoneDirectAir_unit1 Perimeter_ZN_1,    !- Name
    always_avail,             !- Availability Schedule Name
    Zone Inlet Node_unit1 ATInlet Perimeter_ZN_1,    !- Air Inlet Node Name
    Zone Inlet Node_unit1 Perimeter_ZN_1,    !- Air Outlet Node Name
    autosize;                 !- Maximum Air Flow Rate
, 
AirTerminal:SingleDuct:ConstantVolume:NoReheat,
    ZoneDirectAir_unit1 Perimeter_ZN_2,    !- Name
    always_avail,             !- Availability Schedule Name
    Zone Inlet Node_unit1 ATInlet Perimeter_ZN_2,    !- Air Inlet Node Name
    Zone Inlet Node_unit1 Perimeter_ZN_2,    !- Air Outlet N

In [25]:
#Check if the new AirTerminal:SingleDuct:ConstantVolume:NoReheat has the right count same as the conditioned zone count
len(lst_CAV)

5

# Edit ZoneHVAC

In [26]:
#Identify HVAC related AIRTERMINAL object types exist in both files. 
#AIRTERMINAL needs to be capitalized as object types are capitalized in idf files.
zHVAC_target=[s for s in target_HVAC_list if "ZONEHVAC" in s]
zHVAC_source=[s for s in source_HVAC_list if "ZONEHVAC" in s]

zHVAC_both=list(set(zHVAC_target+zHVAC_source))
zHVAC_both

['ZONEHVAC:EQUIPMENTLIST',
 'ZONEHVAC:EQUIPMENTCONNECTIONS',
 'ZONEHVAC:AIRDISTRIBUTIONUNIT']

### ZoneHVAC:AirDistributionUnit

In [27]:
lst_ADU = targetidf.idfobjects['ZoneHVAC:AirDistributionUnit']
lst_ADU

[
ZoneHVAC:AirDistributionUnit,
    Core_ZN Direct Air ADU,    !- Name
    Core_ZN Direct Air Inlet Node Name,    !- Air Distribution Unit Outlet Node Name
    AirTerminal:SingleDuct:ConstantVolume:NoReheat,    !- Air Terminal Object Type
    Core_ZN Direct Air;       !- Air Terminal Name
, 
ZoneHVAC:AirDistributionUnit,
    Perimeter_ZN_1 Direct Air ADU,    !- Name
    Perimeter_ZN_1 Direct Air Inlet Node Name,    !- Air Distribution Unit Outlet Node Name
    AirTerminal:SingleDuct:ConstantVolume:NoReheat,    !- Air Terminal Object Type
    Perimeter_ZN_1 Direct Air;    !- Air Terminal Name
, 
ZoneHVAC:AirDistributionUnit,
    Perimeter_ZN_2 Direct Air ADU,    !- Name
    Perimeter_ZN_2 Direct Air Inlet Node Name,    !- Air Distribution Unit Outlet Node Name
    AirTerminal:SingleDuct:ConstantVolume:NoReheat,    !- Air Terminal Object Type
    Perimeter_ZN_2 Direct Air;    !- Air Terminal Name
, 
ZoneHVAC:AirDistributionUnit,
    Perimeter_ZN_3 Direct Air ADU,    !- Name
    Perimeter

In [28]:
targetidf.idfobjects['ZoneHVAC:AirDistributionUnit'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['ZoneHVAC:AirDistributionUnit'][-1]
    targetidf.copyidfobject(newclg)
    lst_ADU[-1].Name=newclg.Name+' '+zones[i]
    lst_ADU[-1].Air_Distribution_Unit_Outlet_Node_Name=lst_CAV[i].Air_Outlet_Node_Name
    lst_ADU[-1].Air_Terminal_Name=lst_CAV[i].Name
    
lst_ADU

[
ZoneHVAC:AirDistributionUnit,
    ZoneDirectAir_unit1 ADU Core_ZN,    !- Name
    Zone Inlet Node_unit1 Core_ZN,    !- Air Distribution Unit Outlet Node Name
    AirTerminal:SingleDuct:ConstantVolume:NoReheat,    !- Air Terminal Object Type
    ZoneDirectAir_unit1 Core_ZN;    !- Air Terminal Name
, 
ZoneHVAC:AirDistributionUnit,
    ZoneDirectAir_unit1 ADU Perimeter_ZN_1,    !- Name
    Zone Inlet Node_unit1 Perimeter_ZN_1,    !- Air Distribution Unit Outlet Node Name
    AirTerminal:SingleDuct:ConstantVolume:NoReheat,    !- Air Terminal Object Type
    ZoneDirectAir_unit1 Perimeter_ZN_1;    !- Air Terminal Name
, 
ZoneHVAC:AirDistributionUnit,
    ZoneDirectAir_unit1 ADU Perimeter_ZN_2,    !- Name
    Zone Inlet Node_unit1 Perimeter_ZN_2,    !- Air Distribution Unit Outlet Node Name
    AirTerminal:SingleDuct:ConstantVolume:NoReheat,    !- Air Terminal Object Type
    ZoneDirectAir_unit1 Perimeter_ZN_2;    !- Air Terminal Name
, 
ZoneHVAC:AirDistributionUnit,
    ZoneDirectAir_unit1

In [29]:
len(lst_ADU)

5

### ZoneHVAC:EquipmentList 

In [30]:
lst_Eqp = targetidf.idfobjects['ZoneHVAC:EquipmentList']
lst_Eqp

[
ZoneHVAC:EquipmentList,
    Core_ZN Equipment,        !- Name
    SequentialLoad,           !- Load Distribution Scheme
    ZoneHVAC:AirDistributionUnit,    !- Zone Equipment 1 Object Type
    Core_ZN Direct Air ADU,    !- Zone Equipment 1 Name
    1,                        !- Zone Equipment 1 Cooling Sequence
    1,                        !- Zone Equipment 1 Heating or NoLoad Sequence
    ,                         !- Zone Equipment 1 Sequential Cooling Fraction Schedule Name
    ;                         !- Zone Equipment 1 Sequential Heating Fraction Schedule Name
, 
ZoneHVAC:EquipmentList,
    Perimeter_ZN_1 Equipment,    !- Name
    SequentialLoad,           !- Load Distribution Scheme
    ZoneHVAC:AirDistributionUnit,    !- Zone Equipment 1 Object Type
    Perimeter_ZN_1 Direct Air ADU,    !- Zone Equipment 1 Name
    1,                        !- Zone Equipment 1 Cooling Sequence
    1,                        !- Zone Equipment 1 Heating or NoLoad Sequence
    ,                  

In [31]:
#The best way to tackle ZoneHVAC:EquipmentList is to eliminate the original objects and replace with the one in the source file and rename the zone equipment
targetidf.idfobjects['ZoneHVAC:EquipmentList'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['ZoneHVAC:EquipmentList'][-1]
    targetidf.copyidfobject(newclg)
    lst_Eqp[-1].Zone_Equipment_1_Name=newclg.Zone_Equipment_1_Name+' '+zones[i]
    #User can keep adding items here if there are more than one zone equipment in the new electrified system
        
lst_Eqp

[
ZoneHVAC:EquipmentList,
    ZoneEquipment_unit1,      !- Name
    SequentialLoad,           !- Load Distribution Scheme
    ZoneHVAC:AirDistributionUnit,    !- Zone Equipment 1 Object Type
    ZoneDirectAir_unit1 ADU Core_ZN,    !- Zone Equipment 1 Name
    1,                        !- Zone Equipment 1 Cooling Sequence
    1,                        !- Zone Equipment 1 Heating or NoLoad Sequence
    ,                         !- Zone Equipment 1 Sequential Cooling Fraction Schedule Name
    ;                         !- Zone Equipment 1 Sequential Heating Fraction Schedule Name
, 
ZoneHVAC:EquipmentList,
    ZoneEquipment_unit1,      !- Name
    SequentialLoad,           !- Load Distribution Scheme
    ZoneHVAC:AirDistributionUnit,    !- Zone Equipment 1 Object Type
    ZoneDirectAir_unit1 ADU Perimeter_ZN_1,    !- Zone Equipment 1 Name
    1,                        !- Zone Equipment 1 Cooling Sequence
    1,                        !- Zone Equipment 1 Heating or NoLoad Sequence
    ,   

In [32]:
len(lst_Eqp)

5

### ZoneHVAC:EquipmentConnections

In [33]:
lst_Eqpcon = targetidf.idfobjects['ZoneHVAC:EquipmentConnections']
lst_Eqpcon

[
ZoneHVAC:EquipmentConnections,
    Core_ZN,                  !- Zone Name
    Core_ZN Equipment,        !- Zone Conditioning Equipment List Name
    Core_ZN Inlet Nodes,      !- Zone Air Inlet Node or NodeList Name
    ,                         !- Zone Air Exhaust Node or NodeList Name
    Core_ZN Air Node,         !- Zone Air Node Name
    Core_ZN Return Air Node Name;    !- Zone Return Air Node or NodeList Name
, 
ZoneHVAC:EquipmentConnections,
    Perimeter_ZN_1,           !- Zone Name
    Perimeter_ZN_1 Equipment,    !- Zone Conditioning Equipment List Name
    Perimeter_ZN_1 Inlet Nodes,    !- Zone Air Inlet Node or NodeList Name
    ,                         !- Zone Air Exhaust Node or NodeList Name
    Perimeter_ZN_1 Air Node,    !- Zone Air Node Name
    Perimeter_ZN_1 Return Air Node Name;    !- Zone Return Air Node or NodeList Name
, 
ZoneHVAC:EquipmentConnections,
    Perimeter_ZN_2,           !- Zone Name
    Perimeter_ZN_2 Equipment,    !- Zone Conditioning Equipment Lis

In [34]:
sourceidf.idfobjects['ZoneHVAC:EquipmentConnections']

[
ZoneHVAC:EquipmentConnections,
    living_unit1,             !- Zone Name
    ZoneEquipment_unit1,      !- Zone Conditioning Equipment List Name
    zone inlet nodes_unit1,    !- Zone Air Inlet Node or NodeList Name
    ,                         !- Zone Air Exhaust Node or NodeList Name
    Zone Node_unit1,          !- Zone Air Node Name
    Zone Outlet Node_unit1;    !- Zone Return Air Node or NodeList Name
]

In [35]:
for i in range(len(zones)):
    new=sourceidf.idfobjects['ZoneHVAC:EquipmentConnections'][-1]
    #Instead of clearing the original EquipmentConnections and copy-pasting from the source file, here we keep the original objects
    #We only replace the components' and node's name
    lst_Eqpcon[i].Zone_Conditioning_Equipment_List_Name = lst_Eqp[i].Name
    lst_Eqpcon[i].Zone_Air_Inlet_Node_or_NodeList_Name=new.Zone_Air_Inlet_Node_or_NodeList_Name+' '+zones[i]
    lst_Eqpcon[i].Zone_Air_Node_Name=new.Zone_Air_Node_Name+' '+zones[i]
    lst_Eqpcon[i].Zone_Return_Air_Node_or_NodeList_Name=new.Zone_Return_Air_Node_or_NodeList_Name+' '+zones[i]
    
    
lst_Eqpcon

[
ZoneHVAC:EquipmentConnections,
    Core_ZN,                  !- Zone Name
    ZoneEquipment_unit1,      !- Zone Conditioning Equipment List Name
    zone inlet nodes_unit1 Core_ZN,    !- Zone Air Inlet Node or NodeList Name
    ,                         !- Zone Air Exhaust Node or NodeList Name
    Zone Node_unit1 Core_ZN,    !- Zone Air Node Name
    Zone Outlet Node_unit1 Core_ZN;    !- Zone Return Air Node or NodeList Name
, 
ZoneHVAC:EquipmentConnections,
    Perimeter_ZN_1,           !- Zone Name
    ZoneEquipment_unit1,      !- Zone Conditioning Equipment List Name
    zone inlet nodes_unit1 Perimeter_ZN_1,    !- Zone Air Inlet Node or NodeList Name
    ,                         !- Zone Air Exhaust Node or NodeList Name
    Zone Node_unit1 Perimeter_ZN_1,    !- Zone Air Node Name
    Zone Outlet Node_unit1 Perimeter_ZN_1;    !- Zone Return Air Node or NodeList Name
, 
ZoneHVAC:EquipmentConnections,
    Perimeter_ZN_2,           !- Zone Name
    ZoneEquipment_unit1,      !- Zone

# Edit Fans

In [36]:
#Identify HVAC related FAN object types exist in both files. 
#FAN needs to be capitalized as object types are capitalized in idf files.
fan_target=[s for s in target_HVAC_list if "FAN" in s]
fan_source=[s for s in source_HVAC_list if "FAN" in s]

#FAN can be different object types in target and source files. Print out the object types in sequence of target and source.
print(fan_target, fan_source)


['FAN:CONSTANTVOLUME'] ['FAN:ONOFF']


In [37]:
#Since the new fans will be Fan:OnOff, we set the fan list in the target file to Fan:OnOff
lst_fan = targetidf.idfobjects['Fan:OnOff']
lst_fan

[]

In [38]:
sourceidf.idfobjects['Fan:OnOff'][-1]


Fan:OnOff,
    Supply Fan_unit1,         !- Name
    always_avail,             !- Availability Schedule Name
    0.50054,                  !- Fan Total Efficiency
    400,                      !- Pressure Rise
    autosize,                 !- Maximum Flow Rate
    0.863,                    !- Motor Efficiency
    1,                        !- Motor In Airstream Fraction
    air loop inlet node_unit1,    !- Air Inlet Node Name
    cooling coil air inlet node_unit1,    !- Air Outlet Node Name
    ,                         !- Fan Power Ratio Function of Speed Ratio Curve Name
    ,                         !- Fan Efficiency Ratio Function of Speed Ratio Curve Name
    Fan Energy;               !- EndUse Subcategory

In [39]:
targetidf.idfobjects['Fan:ConstantVolume'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Fan:OnOff'][-1]
    targetidf.copyidfobject(newclg)
    lst_fan[-1].Name=newclg.Name+' '+zones[i]
    lst_fan[-1].Air_Inlet_Node_Name=newclg.Air_Inlet_Node_Name+' '+zones[i]
    lst_fan[-1].Air_Outlet_Node_Name=newclg.Air_Outlet_Node_Name+' '+zones[i]
    lst_fan[-1].EndUse_Subcategory='Fan Energy'
   

lst_fan

[
Fan:OnOff,
    Supply Fan_unit1 Core_ZN,    !- Name
    always_avail,             !- Availability Schedule Name
    0.50054,                  !- Fan Total Efficiency
    400,                      !- Pressure Rise
    autosize,                 !- Maximum Flow Rate
    0.863,                    !- Motor Efficiency
    1,                        !- Motor In Airstream Fraction
    air loop inlet node_unit1 Core_ZN,    !- Air Inlet Node Name
    cooling coil air inlet node_unit1 Core_ZN,    !- Air Outlet Node Name
    ,                         !- Fan Power Ratio Function of Speed Ratio Curve Name
    ,                         !- Fan Efficiency Ratio Function of Speed Ratio Curve Name
    Fan Energy;               !- EndUse Subcategory
, 
Fan:OnOff,
    Supply Fan_unit1 Perimeter_ZN_1,    !- Name
    always_avail,             !- Availability Schedule Name
    0.50054,                  !- Fan Total Efficiency
    400,                      !- Pressure Rise
    autosize,                 !- Max

In [40]:
len(lst_fan)

5

# Edit Coils

In [41]:
#Identify HVAC related COIL object types exist in both files. 
#COIL needs to be capitalized as object types are capitalized in idf files.
Coil_target=[s for s in target_HVAC_list if "COIL" in s]
Coil_source=[s for s in source_HVAC_list if "COIL" in s]

#Coil can be different object types in target and source files. Print out the object types in sequence of target and source.
print(Coil_target, Coil_source)

['COIL:COOLING:DX:SINGLESPEED', 'COIL:HEATING:FUEL', 'COILSYSTEM:COOLING:DX'] ['COIL:COOLING:DX:SINGLESPEED', 'COIL:HEATING:ELECTRIC', 'COIL:HEATING:DX:SINGLESPEED']


### Edit SingleSpeed DX Cooling Coil

In [42]:
lst_clgcoilsingle = targetidf.idfobjects['Coil:Cooling:DX:SingleSpeed']
lst_clgcoilsingle

[
Coil:Cooling:DX:SingleSpeed,
    PSZ-AC:1_CoolC DXCoil,    !- Name
    ALWAYS_ON,                !- Availability Schedule Name
    AUTOSIZE,                 !- Gross Rated Total Cooling Capacity
    AUTOSIZE,                 !- Gross Rated Sensible Heat Ratio
    3.06719599275285,         !- Gross Rated Cooling COP
    AUTOSIZE,                 !- Rated Air Flow Rate
    ,                         !- 2017 Rated Evaporator Fan Power Per Volume Flow Rate
    ,                         !- 2023 Rated Evaporator Fan Power Per Volume Flow Rate
    PSZ-AC:1_OA-PSZ-AC:1_CoolCNode,    !- Air Inlet Node Name
    PSZ-AC:1_CoolC-PSZ-AC:1_HeatCNode,    !- Air Outlet Node Name
    Cool-Cap-fT,              !- Total Cooling Capacity Function of Temperature Curve Name
    ConstantCubic,            !- Total Cooling Capacity Function of Flow Fraction Curve Name
    Cool-EIR-fT,              !- Energy Input Ratio Function of Temperature Curve Name
    ConstantCubic,            !- Energy Input Ratio Funct

In [43]:
sourceidf.idfobjects['Coil:Cooling:DX:SingleSpeed']

[
Coil:Cooling:DX:SingleSpeed,
    DX Cooling Coil_unit1,    !- Name
    always_avail,             !- Availability Schedule Name
    autosize,                 !- Gross Rated Total Cooling Capacity
    autosize,                 !- Gross Rated Sensible Heat Ratio
    4.45,                     !- Gross Rated Cooling COP
    autosize,                 !- Rated Air Flow Rate
    636,                      !- 2017 Rated Evaporator Fan Power Per Volume Flow Rate
    636,                      !- 2023 Rated Evaporator Fan Power Per Volume Flow Rate
    Cooling Coil Air Inlet Node_unit1,    !- Air Inlet Node Name
    Heating Coil Air Inlet Node_unit1,    !- Air Outlet Node Name
    Cool-Cap-FT,              !- Total Cooling Capacity Function of Temperature Curve Name
    ConstantCubic,            !- Total Cooling Capacity Function of Flow Fraction Curve Name
    Cool-EIR-FT,              !- Energy Input Ratio Function of Temperature Curve Name
    ConstantCubic,            !- Energy Input Ratio Fu

In [44]:
lst_clgcoilsingle = targetidf.idfobjects['Coil:Cooling:DX:SingleSpeed']
targetidf.idfobjects['Coil:Cooling:DX:SingleSpeed'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Coil:Cooling:DX:SingleSpeed'][-1]
    targetidf.copyidfobject(newclg)
    lst_clgcoilsingle[-1].Name=newclg.Name+' '+zones[i]
    lst_clgcoilsingle[-1].Air_Inlet_Node_Name=newclg.Air_Inlet_Node_Name+' '+zones[i]
    lst_clgcoilsingle[-1].Air_Outlet_Node_Name=newclg.Air_Outlet_Node_Name+' '+zones[i]
    
lst_clgcoilsingle

[
Coil:Cooling:DX:SingleSpeed,
    DX Cooling Coil_unit1 Core_ZN,    !- Name
    always_avail,             !- Availability Schedule Name
    autosize,                 !- Gross Rated Total Cooling Capacity
    autosize,                 !- Gross Rated Sensible Heat Ratio
    4.45,                     !- Gross Rated Cooling COP
    autosize,                 !- Rated Air Flow Rate
    636,                      !- 2017 Rated Evaporator Fan Power Per Volume Flow Rate
    636,                      !- 2023 Rated Evaporator Fan Power Per Volume Flow Rate
    Cooling Coil Air Inlet Node_unit1 Core_ZN,    !- Air Inlet Node Name
    Heating Coil Air Inlet Node_unit1 Core_ZN,    !- Air Outlet Node Name
    Cool-Cap-FT,              !- Total Cooling Capacity Function of Temperature Curve Name
    ConstantCubic,            !- Total Cooling Capacity Function of Flow Fraction Curve Name
    Cool-EIR-FT,              !- Energy Input Ratio Function of Temperature Curve Name
    ConstantCubic,            

In [45]:
len(lst_clgcoilsingle)

5

### Edit Heating Coils

Here finally comes the ELECTRIFICATION. The original gas heating coil is replaced by the electric heating coil and the single-speed DX heating coil. Therefore, two types of non-gas heating coils are added.

In [46]:
lst_htgcoilfuel = targetidf.idfobjects['Coil:Heating:Fuel']
lst_htgcoilfuel = lst_htgcoilfuel.clear()
lst_htgcoilfuel

In [47]:
sourceidf.idfobjects['Coil:Heating:Electric'][0]


Coil:Heating:Electric,
    Electric Heating Coil_unit1,    !- Name
    always_avail,             !- Availability Schedule Name
    1,                        !- Efficiency
    autosize,                 !- Nominal Capacity
    DX Heating Coil Outlet Node_unit1,    !- Air Inlet Node Name
    Air Loop Outlet Node_unit1;    !- Air Outlet Node Name

In [48]:
lst_htgcoilelec = targetidf.idfobjects['Coil:Heating:Electric']
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Coil:Heating:Electric'][-1]
    targetidf.copyidfobject(newclg)
    lst_htgcoilelec[-1].Name=newclg.Name+' '+zones[i]
    lst_htgcoilelec[-1].Air_Inlet_Node_Name=newclg.Air_Inlet_Node_Name+' '+zones[i]
    lst_htgcoilelec[-1].Air_Outlet_Node_Name=newclg.Air_Outlet_Node_Name+' '+zones[i]

    
lst_htgcoilelec

[
Coil:Heating:Electric,
    Electric Heating Coil_unit1 Core_ZN,    !- Name
    always_avail,             !- Availability Schedule Name
    1,                        !- Efficiency
    autosize,                 !- Nominal Capacity
    DX Heating Coil Outlet Node_unit1 Core_ZN,    !- Air Inlet Node Name
    Air Loop Outlet Node_unit1 Core_ZN;    !- Air Outlet Node Name
, 
Coil:Heating:Electric,
    Electric Heating Coil_unit1 Perimeter_ZN_1,    !- Name
    always_avail,             !- Availability Schedule Name
    1,                        !- Efficiency
    autosize,                 !- Nominal Capacity
    DX Heating Coil Outlet Node_unit1 Perimeter_ZN_1,    !- Air Inlet Node Name
    Air Loop Outlet Node_unit1 Perimeter_ZN_1;    !- Air Outlet Node Name
, 
Coil:Heating:Electric,
    Electric Heating Coil_unit1 Perimeter_ZN_2,    !- Name
    always_avail,             !- Availability Schedule Name
    1,                        !- Efficiency
    autosize,                 !- Nominal Capaci

In [49]:
len(lst_htgcoilelec)

5

In [50]:
lst_htgcoilsingle = targetidf.idfobjects['Coil:Heating:DX:SingleSpeed']
lst_htgcoilsingle

[]

In [51]:
sourceidf.idfobjects['Coil:Heating:DX:SingleSpeed'][0]


Coil:Heating:DX:SingleSpeed,
    DX Heating Coil_unit1,    !- Name
    always_avail,             !- Availability Schedule Name
    autosize,                 !- Gross Rated Heating Capacity
    4.37,                     !- Gross Rated Heating COP
    autosize,                 !- Rated Air Flow Rate
    ,                         !- 2017 Rated Supply Fan Power Per Volume Flow Rate
    636,                      !- 2023 Rated Supply Fan Power Per Volume Flow Rate
    Heating Coil Air Inlet Node_unit1,    !- Air Inlet Node Name
    DX Heating Coil Outlet Node_unit1,    !- Air Outlet Node Name
    DXHtgCoilTotalHtgCapFuncTemperature,    !- Heating Capacity Function of Temperature Curve Name
    DXHtgCoilTotalHtgCapFuncFlowFraction,    !- Heating Capacity Function of Flow Fraction Curve Name
    DXHtgCoilEnergyInputRatioFuncTemperature,    !- Energy Input Ratio Function of Temperature Curve Name
    DXHtgCoilEnergyInputRatioFuncFlowFraction,    !- Energy Input Ratio Function of Flow Fraction 

In [52]:
lst_htgcoilsingle = targetidf.idfobjects['Coil:Heating:DX:SingleSpeed']
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Coil:Heating:DX:SingleSpeed'][-1]
    targetidf.copyidfobject(newclg)
    lst_htgcoilsingle[-1].Name=newclg.Name+' '+zones[i]
    lst_htgcoilsingle[-1].Air_Inlet_Node_Name=newclg.Air_Inlet_Node_Name+' '+zones[i]
    lst_htgcoilsingle[-1].Air_Outlet_Node_Name=newclg.Air_Outlet_Node_Name+' '+zones[i]
    
    
lst_htgcoilsingle

[
Coil:Heating:DX:SingleSpeed,
    DX Heating Coil_unit1 Core_ZN,    !- Name
    always_avail,             !- Availability Schedule Name
    autosize,                 !- Gross Rated Heating Capacity
    4.37,                     !- Gross Rated Heating COP
    autosize,                 !- Rated Air Flow Rate
    ,                         !- 2017 Rated Supply Fan Power Per Volume Flow Rate
    636,                      !- 2023 Rated Supply Fan Power Per Volume Flow Rate
    Heating Coil Air Inlet Node_unit1 Core_ZN,    !- Air Inlet Node Name
    DX Heating Coil Outlet Node_unit1 Core_ZN,    !- Air Outlet Node Name
    DXHtgCoilTotalHtgCapFuncTemperature,    !- Heating Capacity Function of Temperature Curve Name
    DXHtgCoilTotalHtgCapFuncFlowFraction,    !- Heating Capacity Function of Flow Fraction Curve Name
    DXHtgCoilEnergyInputRatioFuncTemperature,    !- Energy Input Ratio Function of Temperature Curve Name
    DXHtgCoilEnergyInputRatioFuncFlowFraction,    !- Energy Input Ratio F

Before editing the entire AirLoopHVAC, we address some other components like Controller, OutdoorAir, etc. first.

# Edit Controller

In [53]:
#Identify HVAC related CONTROLLER object types exist in both files. 
#CONTROLLER needs to be capitalized as object types are capitalized in idf files.
ctr_target=[s for s in target_HVAC_list if "CONTROLLER" in s]
ctr_source=[s for s in source_HVAC_list if "CONTROLLER" in s]

#Coil can be different object types in target and source files. Print out the object types in sequence of target and source.
print(ctr_target, ctr_source)

['CONTROLLER:OUTDOORAIR', 'AIRLOOPHVAC:CONTROLLERLIST'] []


Since the controller related list is empty for the source file, that means in the electrified HVAC system, there's no controller related items needed at this point. Since we already cleaned all unnecessary objects at the beginning, we don't need to make any modifications here. If any controller related items is wanted, it could be either addressed after the target file is fully electrified, or addressed already in the source file as part of the reference.

# Edit OutdoorAir

In [54]:
#Identify HVAC related OUTDOORAIR: object types exist in both files. 
#OUTDOORAIR: needs to be capitalized as object types are capitalized in idf files.
oa_target=[s for s in target_HVAC_list if "OUTDOORAIR:" in s]
oa_source=[s for s in source_HVAC_list if "OUTDOORAIR:" in s]

#Coil can be different object types in target and source files. Print out the object types in sequence of target and source.
print(oa_target, oa_source)

['OUTDOORAIR:MIXER', 'OUTDOORAIR:NODE', 'OUTDOORAIR:NODELIST'] []


Same as Controller, no actions needed here as all unneccessary OutdoorAir objects have been cleared already.

# Edit AirLoopHVAC

In [55]:
#Identify HVAC related AIRLOOPHVAC object types exist in both files. 
#AIRLOOPHVAC needs to be capitalized as object types are capitalized in idf files.
ALHVAC_target=[s for s in target_HVAC_list if "AIRLOOPHVAC" in s]
ALHVAC_source=[s for s in source_HVAC_list if "AIRLOOPHVAC" in s]

#AirLoopHVAC can be different object types in target and source files. Print out the object types in sequence of target and source.
print(ALHVAC_target, ALHVAC_source)

['AIRLOOPHVAC:CONTROLLERLIST', 'AIRLOOPHVAC', 'AIRLOOPHVAC:OUTDOORAIRSYSTEM:EQUIPMENTLIST', 'AIRLOOPHVAC:OUTDOORAIRSYSTEM', 'AIRLOOPHVAC:ZONESPLITTER', 'AIRLOOPHVAC:SUPPLYPATH', 'AIRLOOPHVAC:ZONEMIXER', 'AIRLOOPHVAC:RETURNPATH'] ['AIRLOOPHVAC:UNITARYHEATPUMP:AIRTOAIR', 'AIRLOOPHVAC', 'AIRLOOPHVAC:ZONESPLITTER', 'AIRLOOPHVAC:SUPPLYPATH', 'AIRLOOPHVAC:ZONEMIXER', 'AIRLOOPHVAC:RETURNPATH']


Here we have a long list of AirLoopHVAC object types. We start with object types exist in the source file. Any object types that exist in the target file but not the source file have already be cleared. Here we focus on what needs to be added and replaced. We also do not address the object type AirLoopHVAC in this section and prioritize other related components first, as the object type AirLoopHVAC involves object types like Branch and Nodes, which have not been addressed yet.

### Edit AirLoopHVAC:UnitaryHeatPump:AirToAir

In [56]:
lst_unihp = targetidf.idfobjects['AirLoopHVAC:UnitaryHeatPump:AirToAir']
lst_unihp

[]

In [57]:
sourceidf.idfobjects['AirLoopHVAC:UnitaryHeatPump:AirToAir']

[
AirLoopHVAC:UnitaryHeatPump:AirToAir,
    Unitary HP AirToAir living_unit1,    !- Name
    always_avail,             !- Availability Schedule Name
    air loop inlet node_unit1,    !- Air Inlet Node Name
    air loop outlet node_unit1,    !- Air Outlet Node Name
    autosize,                 !- Cooling Supply Air Flow Rate
    autosize,                 !- Heating Supply Air Flow Rate
    0,                        !- No Load Supply Air Flow Rate
    living_unit1,             !- Controlling Zone or Thermostat Location
    Fan:OnOff,                !- Supply Air Fan Object Type
    Supply Fan_unit1,         !- Supply Air Fan Name
    Coil:Heating:DX:SingleSpeed,    !- Heating Coil Object Type
    DX Heating Coil_unit1,    !- Heating Coil Name
    Coil:Cooling:DX:SingleSpeed,    !- Cooling Coil Object Type
    DX Cooling Coil_unit1,    !- Cooling Coil Name
    Coil:Heating:Electric,    !- Supplemental Heating Coil Object Type
    Electric Heating Coil_unit1,    !- Supplemental Heating Co

For the unitary heatpump air loop, instead of following the previous naming replacement logic, we set the coils name to the previously defined coils objects to avoid potential miss match.

In [58]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['AirLoopHVAC:UnitaryHeatPump:AirToAir'][0]
    targetidf.copyidfobject(newclg)
    lst_unihp[-1].Name=newclg.Name+' '+zones[i]
    lst_unihp[-1].Controlling_Zone_or_Thermostat_Location=zones[i]
    
    lst_unihp[-1].Supply_Air_Fan_Name=lst_fan[i].Name
    lst_unihp[-1].Heating_Coil_Name=lst_htgcoilsingle[i].Name#set the heating coil name same as the single-speed DX heating coil
    lst_unihp[-1].Cooling_Coil_Name=lst_clgcoilsingle[i].Name#set the cooling coil name same as the single-speed DX cooling coil
    lst_unihp[-1].Supplemental_Heating_Coil_Name=lst_htgcoilelec[i].Name#set the supplement heating coil name same as the electric heating coil

    lst_unihp[-1].Air_Inlet_Node_Name=newclg.Air_Inlet_Node_Name+' '+zones[i]
    lst_unihp[-1].Air_Outlet_Node_Name=newclg.Air_Outlet_Node_Name+' '+zones[i]
      
    
 
    
lst_unihp

[
AirLoopHVAC:UnitaryHeatPump:AirToAir,
    Unitary HP AirToAir living_unit1 Core_ZN,    !- Name
    always_avail,             !- Availability Schedule Name
    air loop inlet node_unit1 Core_ZN,    !- Air Inlet Node Name
    air loop outlet node_unit1 Core_ZN,    !- Air Outlet Node Name
    autosize,                 !- Cooling Supply Air Flow Rate
    autosize,                 !- Heating Supply Air Flow Rate
    0,                        !- No Load Supply Air Flow Rate
    Core_ZN,                  !- Controlling Zone or Thermostat Location
    Fan:OnOff,                !- Supply Air Fan Object Type
    Supply Fan_unit1 Core_ZN,    !- Supply Air Fan Name
    Coil:Heating:DX:SingleSpeed,    !- Heating Coil Object Type
    DX Heating Coil_unit1 Core_ZN,    !- Heating Coil Name
    Coil:Cooling:DX:SingleSpeed,    !- Cooling Coil Object Type
    DX Cooling Coil_unit1 Core_ZN,    !- Cooling Coil Name
    Coil:Heating:Electric,    !- Supplemental Heating Coil Object Type
    Electric Heatin

In [59]:
len(lst_unihp)

5

### Edit Zone Splitter

In [60]:
lst_splitter = targetidf.idfobjects['AirLoopHVAC:ZoneSplitter']
lst_splitter

[
AirLoopHVAC:ZoneSplitter,
    PSZ-AC:1 Supply Air Splitter,    !- Name
    PSZ-AC:1 Zone Equipment Inlet Node,    !- Inlet Node Name
    Core_ZN Direct Air Inlet Node Name ATInlet;    !- Outlet 1 Node Name
, 
AirLoopHVAC:ZoneSplitter,
    PSZ-AC:2 Supply Air Splitter,    !- Name
    PSZ-AC:2 Zone Equipment Inlet Node,    !- Inlet Node Name
    Perimeter_ZN_1 Direct Air Inlet Node Name ATInlet;    !- Outlet 1 Node Name
, 
AirLoopHVAC:ZoneSplitter,
    PSZ-AC:3 Supply Air Splitter,    !- Name
    PSZ-AC:3 Zone Equipment Inlet Node,    !- Inlet Node Name
    Perimeter_ZN_2 Direct Air Inlet Node Name ATInlet;    !- Outlet 1 Node Name
, 
AirLoopHVAC:ZoneSplitter,
    PSZ-AC:4 Supply Air Splitter,    !- Name
    PSZ-AC:4 Zone Equipment Inlet Node,    !- Inlet Node Name
    Perimeter_ZN_3 Direct Air Inlet Node Name ATInlet;    !- Outlet 1 Node Name
, 
AirLoopHVAC:ZoneSplitter,
    PSZ-AC:5 Supply Air Splitter,    !- Name
    PSZ-AC:5 Zone Equipment Inlet Node,    !- Inlet Node Name
    Peri

In [61]:
sourceidf.idfobjects['AirLoopHVAC:ZoneSplitter']

[
AirLoopHVAC:ZoneSplitter,
    Zone Supply Air Splitter_unit1,    !- Name
    Zone Equipment Inlet Node_unit1,    !- Inlet Node Name
    Zone Inlet Node_unit1 ATInlet;    !- Outlet 1 Node Name
]

In [62]:
targetidf.idfobjects['AirLoopHVAC:ZoneSplitter'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['AirLoopHVAC:ZoneSplitter'][-1]
    targetidf.copyidfobject(newclg)
    lst_splitter[-1].Name=newclg.Name+' '+zones[i]
    lst_splitter[-1].Inlet_Node_Name=newclg.Inlet_Node_Name+' '+zones[i]
    lst_splitter[-1].Outlet_1_Node_Name=newclg.Outlet_1_Node_Name+' '+zones[i]
     
        
lst_splitter

[
AirLoopHVAC:ZoneSplitter,
    Zone Supply Air Splitter_unit1 Core_ZN,    !- Name
    Zone Equipment Inlet Node_unit1 Core_ZN,    !- Inlet Node Name
    Zone Inlet Node_unit1 ATInlet Core_ZN;    !- Outlet 1 Node Name
, 
AirLoopHVAC:ZoneSplitter,
    Zone Supply Air Splitter_unit1 Perimeter_ZN_1,    !- Name
    Zone Equipment Inlet Node_unit1 Perimeter_ZN_1,    !- Inlet Node Name
    Zone Inlet Node_unit1 ATInlet Perimeter_ZN_1;    !- Outlet 1 Node Name
, 
AirLoopHVAC:ZoneSplitter,
    Zone Supply Air Splitter_unit1 Perimeter_ZN_2,    !- Name
    Zone Equipment Inlet Node_unit1 Perimeter_ZN_2,    !- Inlet Node Name
    Zone Inlet Node_unit1 ATInlet Perimeter_ZN_2;    !- Outlet 1 Node Name
, 
AirLoopHVAC:ZoneSplitter,
    Zone Supply Air Splitter_unit1 Perimeter_ZN_3,    !- Name
    Zone Equipment Inlet Node_unit1 Perimeter_ZN_3,    !- Inlet Node Name
    Zone Inlet Node_unit1 ATInlet Perimeter_ZN_3;    !- Outlet 1 Node Name
, 
AirLoopHVAC:ZoneSplitter,
    Zone Supply Air Splitter_unit

In [63]:
len(lst_splitter)

5

### Edit Supply Path 

In [64]:
lst_splypath = targetidf.idfobjects['AirLoopHVAC:SupplyPath']
lst_splypath

[
AirLoopHVAC:SupplyPath,
    PSZ-AC:1,                 !- Name
    PSZ-AC:1 Zone Equipment Inlet Node,    !- Supply Air Path Inlet Node Name
    AirLoopHVAC:ZoneSplitter,    !- Component 1 Object Type
    PSZ-AC:1 Supply Air Splitter;    !- Component 1 Name
, 
AirLoopHVAC:SupplyPath,
    PSZ-AC:2,                 !- Name
    PSZ-AC:2 Zone Equipment Inlet Node,    !- Supply Air Path Inlet Node Name
    AirLoopHVAC:ZoneSplitter,    !- Component 1 Object Type
    PSZ-AC:2 Supply Air Splitter;    !- Component 1 Name
, 
AirLoopHVAC:SupplyPath,
    PSZ-AC:3,                 !- Name
    PSZ-AC:3 Zone Equipment Inlet Node,    !- Supply Air Path Inlet Node Name
    AirLoopHVAC:ZoneSplitter,    !- Component 1 Object Type
    PSZ-AC:3 Supply Air Splitter;    !- Component 1 Name
, 
AirLoopHVAC:SupplyPath,
    PSZ-AC:4,                 !- Name
    PSZ-AC:4 Zone Equipment Inlet Node,    !- Supply Air Path Inlet Node Name
    AirLoopHVAC:ZoneSplitter,    !- Component 1 Object Type
    PSZ-AC:4 Suppl

In [65]:
sourceidf.idfobjects['AirLoopHVAC:SupplyPath']

[
AirLoopHVAC:SupplyPath,
    SupplyPath_unit1,         !- Name
    Zone Equipment Inlet Node_unit1,    !- Supply Air Path Inlet Node Name
    AirLoopHVAC:ZoneSplitter,    !- Component 1 Object Type
    Zone Supply Air Splitter_unit1;    !- Component 1 Name
]

In [66]:
targetidf.idfobjects['AirLoopHVAC:SupplyPath'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['AirLoopHVAC:SupplyPath'][-1]
    targetidf.copyidfobject(newclg)
    lst_splypath[-1].Name=newclg.Name+' '+zones[i]
    lst_splypath[-1].Supply_Air_Path_Inlet_Node_Name=newclg.Supply_Air_Path_Inlet_Node_Name+' '+zones[i]
    lst_splypath[-1].Component_1_Name=lst_splitter[i].Name#Refer to the previous corresponding component, which is zone splitter in this case


lst_splypath

[
AirLoopHVAC:SupplyPath,
    SupplyPath_unit1 Core_ZN,    !- Name
    Zone Equipment Inlet Node_unit1 Core_ZN,    !- Supply Air Path Inlet Node Name
    AirLoopHVAC:ZoneSplitter,    !- Component 1 Object Type
    Zone Supply Air Splitter_unit1 Core_ZN;    !- Component 1 Name
, 
AirLoopHVAC:SupplyPath,
    SupplyPath_unit1 Perimeter_ZN_1,    !- Name
    Zone Equipment Inlet Node_unit1 Perimeter_ZN_1,    !- Supply Air Path Inlet Node Name
    AirLoopHVAC:ZoneSplitter,    !- Component 1 Object Type
    Zone Supply Air Splitter_unit1 Perimeter_ZN_1;    !- Component 1 Name
, 
AirLoopHVAC:SupplyPath,
    SupplyPath_unit1 Perimeter_ZN_2,    !- Name
    Zone Equipment Inlet Node_unit1 Perimeter_ZN_2,    !- Supply Air Path Inlet Node Name
    AirLoopHVAC:ZoneSplitter,    !- Component 1 Object Type
    Zone Supply Air Splitter_unit1 Perimeter_ZN_2;    !- Component 1 Name
, 
AirLoopHVAC:SupplyPath,
    SupplyPath_unit1 Perimeter_ZN_3,    !- Name
    Zone Equipment Inlet Node_unit1 Perimeter_ZN_3

In [67]:
len(lst_splypath)

5

### Edit Mixer

In [68]:
lst_mixer = targetidf.idfobjects['AirLoopHVAC:ZoneMixer']
lst_mixer

[
AirLoopHVAC:ZoneMixer,
    PSZ-AC:1 Return Air Mixer,    !- Name
    PSZ-AC:1 Zone Equipment Outlet Node,    !- Outlet Node Name
    Core_ZN Return Air Node Name;    !- Inlet 1 Node Name
, 
AirLoopHVAC:ZoneMixer,
    PSZ-AC:2 Return Air Mixer,    !- Name
    PSZ-AC:2 Zone Equipment Outlet Node,    !- Outlet Node Name
    Perimeter_ZN_1 Return Air Node Name;    !- Inlet 1 Node Name
, 
AirLoopHVAC:ZoneMixer,
    PSZ-AC:3 Return Air Mixer,    !- Name
    PSZ-AC:3 Zone Equipment Outlet Node,    !- Outlet Node Name
    Perimeter_ZN_2 Return Air Node Name;    !- Inlet 1 Node Name
, 
AirLoopHVAC:ZoneMixer,
    PSZ-AC:4 Return Air Mixer,    !- Name
    PSZ-AC:4 Zone Equipment Outlet Node,    !- Outlet Node Name
    Perimeter_ZN_3 Return Air Node Name;    !- Inlet 1 Node Name
, 
AirLoopHVAC:ZoneMixer,
    PSZ-AC:5 Return Air Mixer,    !- Name
    PSZ-AC:5 Zone Equipment Outlet Node,    !- Outlet Node Name
    Perimeter_ZN_4 Return Air Node Name;    !- Inlet 1 Node Name
]

In [69]:
sourceidf.idfobjects['AirLoopHVAC:ZoneMixer']

[
AirLoopHVAC:ZoneMixer,
    Zone Return Air Mixer_unit1,    !- Name
    Return Air Mixer Outlet_unit1,    !- Outlet Node Name
    Zone Outlet Node_unit1;    !- Inlet 1 Node Name
]

In [70]:
targetidf.idfobjects['AirLoopHVAC:ZoneMixer'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['AirLoopHVAC:ZoneMixer'][-1]
    targetidf.copyidfobject(newclg)
    lst_mixer[-1].Name=newclg.Name+' '+zones[i]
    lst_mixer[-1].Outlet_Node_Name=newclg.Outlet_Node_Name+' '+zones[i]
    lst_mixer[-1].Inlet_1_Node_Name=newclg.Inlet_1_Node_Name+' '+zones[i]
     
  
    
lst_mixer

[
AirLoopHVAC:ZoneMixer,
    Zone Return Air Mixer_unit1 Core_ZN,    !- Name
    Return Air Mixer Outlet_unit1 Core_ZN,    !- Outlet Node Name
    Zone Outlet Node_unit1 Core_ZN;    !- Inlet 1 Node Name
, 
AirLoopHVAC:ZoneMixer,
    Zone Return Air Mixer_unit1 Perimeter_ZN_1,    !- Name
    Return Air Mixer Outlet_unit1 Perimeter_ZN_1,    !- Outlet Node Name
    Zone Outlet Node_unit1 Perimeter_ZN_1;    !- Inlet 1 Node Name
, 
AirLoopHVAC:ZoneMixer,
    Zone Return Air Mixer_unit1 Perimeter_ZN_2,    !- Name
    Return Air Mixer Outlet_unit1 Perimeter_ZN_2,    !- Outlet Node Name
    Zone Outlet Node_unit1 Perimeter_ZN_2;    !- Inlet 1 Node Name
, 
AirLoopHVAC:ZoneMixer,
    Zone Return Air Mixer_unit1 Perimeter_ZN_3,    !- Name
    Return Air Mixer Outlet_unit1 Perimeter_ZN_3,    !- Outlet Node Name
    Zone Outlet Node_unit1 Perimeter_ZN_3;    !- Inlet 1 Node Name
, 
AirLoopHVAC:ZoneMixer,
    Zone Return Air Mixer_unit1 Perimeter_ZN_4,    !- Name
    Return Air Mixer Outlet_unit1 Per

In [71]:
len(lst_mixer)

5

### Edit Return Path

In [72]:
lst_rtnpath = targetidf.idfobjects['AirLoopHVAC:ReturnPath']
lst_rtnpath

[
AirLoopHVAC:ReturnPath,
    PSZ-AC:1 Return Air Path,    !- Name
    PSZ-AC:1 Zone Equipment Outlet Node,    !- Return Air Path Outlet Node Name
    AirLoopHVAC:ZoneMixer,    !- Component 1 Object Type
    PSZ-AC:1 Return Air Mixer;    !- Component 1 Name
, 
AirLoopHVAC:ReturnPath,
    PSZ-AC:2 Return Air Path,    !- Name
    PSZ-AC:2 Zone Equipment Outlet Node,    !- Return Air Path Outlet Node Name
    AirLoopHVAC:ZoneMixer,    !- Component 1 Object Type
    PSZ-AC:2 Return Air Mixer;    !- Component 1 Name
, 
AirLoopHVAC:ReturnPath,
    PSZ-AC:3 Return Air Path,    !- Name
    PSZ-AC:3 Zone Equipment Outlet Node,    !- Return Air Path Outlet Node Name
    AirLoopHVAC:ZoneMixer,    !- Component 1 Object Type
    PSZ-AC:3 Return Air Mixer;    !- Component 1 Name
, 
AirLoopHVAC:ReturnPath,
    PSZ-AC:4 Return Air Path,    !- Name
    PSZ-AC:4 Zone Equipment Outlet Node,    !- Return Air Path Outlet Node Name
    AirLoopHVAC:ZoneMixer,    !- Component 1 Object Type
    PSZ-AC:4 Return

In [73]:
sourceidf.idfobjects['AirLoopHVAC:ReturnPath']

[
AirLoopHVAC:ReturnPath,
    ReturnPath_unit1,         !- Name
    Return Air Mixer Outlet_unit1,    !- Return Air Path Outlet Node Name
    AirLoopHVAC:ZoneMixer,    !- Component 1 Object Type
    Zone Return Air Mixer_unit1;    !- Component 1 Name
]

In [74]:
targetidf.idfobjects['AirLoopHVAC:ReturnPath'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['AirLoopHVAC:ReturnPath'][-1]
    targetidf.copyidfobject(newclg)
    lst_rtnpath[-1].Name=newclg.Name+' '+zones[i]
    lst_rtnpath[-1].Return_Air_Path_Outlet_Node_Name=newclg.Return_Air_Path_Outlet_Node_Name+' '+zones[i]
    lst_rtnpath[-1].Component_1_Name=lst_mixer[i].Name#Refer to the previous corresponding component, which is zone mixer in this case
     
   
    
lst_rtnpath

[
AirLoopHVAC:ReturnPath,
    ReturnPath_unit1 Core_ZN,    !- Name
    Return Air Mixer Outlet_unit1 Core_ZN,    !- Return Air Path Outlet Node Name
    AirLoopHVAC:ZoneMixer,    !- Component 1 Object Type
    Zone Return Air Mixer_unit1 Core_ZN;    !- Component 1 Name
, 
AirLoopHVAC:ReturnPath,
    ReturnPath_unit1 Perimeter_ZN_1,    !- Name
    Return Air Mixer Outlet_unit1 Perimeter_ZN_1,    !- Return Air Path Outlet Node Name
    AirLoopHVAC:ZoneMixer,    !- Component 1 Object Type
    Zone Return Air Mixer_unit1 Perimeter_ZN_1;    !- Component 1 Name
, 
AirLoopHVAC:ReturnPath,
    ReturnPath_unit1 Perimeter_ZN_2,    !- Name
    Return Air Mixer Outlet_unit1 Perimeter_ZN_2,    !- Return Air Path Outlet Node Name
    AirLoopHVAC:ZoneMixer,    !- Component 1 Object Type
    Zone Return Air Mixer_unit1 Perimeter_ZN_2;    !- Component 1 Name
, 
AirLoopHVAC:ReturnPath,
    ReturnPath_unit1 Perimeter_ZN_3,    !- Name
    Return Air Mixer Outlet_unit1 Perimeter_ZN_3,    !- Return Air Path

In [75]:
len(lst_rtnpath)

5

# Edit Branch

Finally there we come, the branch and the node. We need to first locate where the HVAC branches are located in the target file. Make sure the naming of the air loop branch in the target file is clear and discernable, and all air loop branches are arranged next to each other. We also need to repeat the same process of HVAC branches in the source file as well, though we will only use one of them to do the duplication.

Here we define a function to locate the HVAC branches.

In [76]:
def find_AC_branch(keywords, idf, obj_type):
    lst_objtype=idf.idfobjects[obj_type] #Pull out all branches into a list for an idf
    lst_objtype_name=[]
    
    #check if any of the keywords exist in the branch name
    for i in range(len(lst_objtype)):
        name=lst_objtype[i].Name
        if any(keyword in name for keyword in keywords):
            lst_objtype_name.append(True)
        else:
            lst_objtype_name.append(False)
    
    #Get a list of indices of AC branches in the idf
    lst_AC_objtype_idx=[i for i,x in enumerate(lst_objtype_name) if x] 
    return lst_AC_objtype_idx

Here the keywords to identify HVAC air loop branch are "AC" and "Air" - this can be edit by user.

In [77]:
keys=['AC', 'Air'] #Use defined keywords
lst_target_AC_branch_idx=find_AC_branch(keys,targetidf,'Branch')
lst_target_AC_branch_idx

[0, 1, 2, 3, 4]

Repeat the same process for the source file.

In [78]:
lst_source_AC_branch_idx=find_AC_branch(keys,sourceidf,'Branch')
lst_source_AC_branch_idx

[0]

Woohoo. The goal here is to replace the original HVAC branches in the target file with ONE HVAC branch from the source file following the same naming logic introduced earlier. We will use the FIRST HVAC branch in the source file to avoid any complexities.

We still need to first define lst_branch to be editable.

In [79]:
lst_branch = targetidf.idfobjects['Branch']

In [80]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Branch'][lst_source_AC_branch_idx[0]]
    targetidf.copyidfobject(newclg)
    lst_branch[-1].Name=newclg.Name+' '+zones[i]
      
    lst_branch[-1].Component_1_Object_Type='AirLoopHVAC:UnitaryHeatPump:AirToAir' #This can be modified depends on the source HVAC type
    lst_branch[-1].Component_1_Name=lst_unihp[i].Name #Same. Here we specifically connect it to the component name listed earlier. This should be changed depends on the source file HVAC type.
    lst_branch[-1].Component_1_Inlet_Node_Name = newclg.Component_1_Inlet_Node_Name+' '+zones[i]
    lst_branch[-1].Component_1_Outlet_Node_Name = newclg.Component_1_Outlet_Node_Name+' '+zones[i]
    
    
lst_branch

[
Branch,
    PSZ-AC:1 Air Loop Main Branch,    !- Name
    ,                         !- Pressure Drop Curve Name
    AirLoopHVAC:OutdoorAirSystem,    !- Component 1 Object Type
    PSZ-AC:1_OA,              !- Component 1 Name
    PSZ-AC:1 Supply Equipment Inlet Node,    !- Component 1 Inlet Node Name
    PSZ-AC:1_OA-PSZ-AC:1_CoolCNode,    !- Component 1 Outlet Node Name
    CoilSystem:Cooling:DX,    !- Component 2 Object Type
    PSZ-AC:1_CoolC,           !- Component 2 Name
    PSZ-AC:1_OA-PSZ-AC:1_CoolCNode,    !- Component 2 Inlet Node Name
    PSZ-AC:1_CoolC-PSZ-AC:1_HeatCNode,    !- Component 2 Outlet Node Name
    Coil:Heating:Fuel,        !- Component 3 Object Type
    PSZ-AC:1_HeatC,           !- Component 3 Name
    PSZ-AC:1_CoolC-PSZ-AC:1_HeatCNode,    !- Component 3 Inlet Node Name
    PSZ-AC:1_HeatC-PSZ-AC:1_FanNode,    !- Component 3 Outlet Node Name
    Fan:ConstantVolume,       !- Component 4 Object Type
    PSZ-AC:1_Fan,             !- Component 4 Name
    PSZ-AC:1_He

In [81]:
len(lst_branch)#Now instead of the original 13 branches, 5 more new HVAC branches are added

18

After duplicating the new branches, we need to remove the original HVAC branches. 

In [82]:
for i in lst_target_AC_branch_idx: #Use the previously identified indices of target file original HVAC branches
    targetidf.idfobjects['Branch'].pop(0)
    
len(targetidf.idfobjects['Branch']) #The number of total branches should stay the same as original after the replacement.

13

In [83]:
#Double check if the old branches are removed and the new branches are added in the end
targetidf.idfobjects['Branch']

[
Branch,
    SWHSys1 Demand Bypass Branch,    !- Name
    ,                         !- Pressure Drop Curve Name
    Pipe:Adiabatic,           !- Component 1 Object Type
    SWHSys1 Demand Bypass Pipe,    !- Component 1 Name
    SWHSys1 Demand Bypass Pipe Inlet Node,    !- Component 1 Inlet Node Name
    SWHSys1 Demand Bypass Pipe Outlet Node;    !- Component 1 Outlet Node Name
, 
Branch,
    SWHSys1 Demand Inlet Branch,    !- Name
    ,                         !- Pressure Drop Curve Name
    Pipe:Adiabatic,           !- Component 1 Object Type
    SWHSys1 Demand Inlet Pipe,    !- Component 1 Name
    SWHSys1 Demand Inlet Node,    !- Component 1 Inlet Node Name
    SWHSys1 Demand Inlet Pipe-SWHSys1 Demand Mixer;    !- Component 1 Outlet Node Name
, 
Branch,
    SWHSys1 Demand Load Branch 1,    !- Name
    ,                         !- Pressure Drop Curve Name
    WaterUse:Connections,     !- Component 1 Object Type
    Core_ZN Water Equipment,    !- Component 1 Name
    Core_ZN Water Eq

### Branch List

Branch List can be edited using the same method.

In [84]:
lst_target_AC_branchlist_idx=find_AC_branch(keys,targetidf,'BranchList')
lst_target_AC_branchlist_idx

[0, 1, 2, 3, 4]

In [85]:
lst_source_AC_branchlist_idx=find_AC_branch(keys,sourceidf,'BranchList')
lst_source_AC_branchlist_idx

[0]

In [86]:
lst_branchlst = targetidf.idfobjects['BranchList']

In [87]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['BranchList'][lst_source_AC_branchlist_idx[0]]
    targetidf.copyidfobject(newclg)
    lst_branchlst[-1].Name=newclg.Name+' '+zones[i]
    lst_branchlst[-1].Branch_1_Name=newclg.Branch_1_Name+' '+zones[i]
        
     
lst_branchlst

[
BranchList,
    PSZ-AC:1 Air Loop Branches,    !- Name
    PSZ-AC:1 Air Loop Main Branch;    !- Branch 1 Name
, 
BranchList,
    PSZ-AC:2 Air Loop Branches,    !- Name
    PSZ-AC:2 Air Loop Main Branch;    !- Branch 1 Name
, 
BranchList,
    PSZ-AC:3 Air Loop Branches,    !- Name
    PSZ-AC:3 Air Loop Main Branch;    !- Branch 1 Name
, 
BranchList,
    PSZ-AC:4 Air Loop Branches,    !- Name
    PSZ-AC:4 Air Loop Main Branch;    !- Branch 1 Name
, 
BranchList,
    PSZ-AC:5 Air Loop Branches,    !- Name
    PSZ-AC:5 Air Loop Main Branch;    !- Branch 1 Name
, 
BranchList,
    SWHSys1 Demand Branches,    !- Name
    SWHSys1 Demand Inlet Branch,    !- Branch 1 Name
    SWHSys1 Demand Load Branch 1,    !- Branch 2 Name
    SWHSys1 Demand Bypass Branch,    !- Branch 3 Name
    SWHSys1 Demand Outlet Branch;    !- Branch 4 Name
, 
BranchList,
    SWHSys1 Supply Branches,    !- Name
    SWHSys1 Supply Inlet Branch,    !- Branch 1 Name
    SWHSys1 Supply Equipment Branch,    !- Branch 2 Name
 

In [88]:
for i in lst_target_AC_branchlist_idx:
    targetidf.idfobjects['BranchList'].pop(0)
    
len(targetidf.idfobjects['BranchList'])

7

### Node List

Node List can be editted using the same method. A few keywords are added here - Inlet, Outlet, etc..

In [89]:
node_keys=['AC', 'Air','Inlet','Outlet'] #Use defined keywords
lst_target_AC_nodelst_idx=find_AC_branch(node_keys,targetidf,'NodeList')
lst_target_AC_nodelst_idx

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [90]:
lst_source_AC_nodelst_idx=find_AC_branch(node_keys,sourceidf,'NodeList')
lst_source_AC_nodelst_idx

[0]

In [91]:
lst_ndlst = targetidf.idfobjects['NodeList']
lst_ndlst

[
NodeList,
    Core_ZN Inlet Nodes,      !- Name
    Core_ZN Direct Air Inlet Node Name;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:1_OANode List,     !- Name
    PSZ-AC:1_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:2_OANode List,     !- Name
    PSZ-AC:2_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:3_OANode List,     !- Name
    PSZ-AC:3_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:4_OANode List,     !- Name
    PSZ-AC:4_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:5_OANode List,     !- Name
    PSZ-AC:5_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    Perimeter_ZN_1 Inlet Nodes,    !- Name
    Perimeter_ZN_1 Direct Air Inlet Node Name;    !- Node 1 Name
, 
NodeList,
    Perimeter_ZN_2 Inlet Nodes,    !- Name
    Perimeter_ZN_2 Direct Air Inlet Node Name;    !- Node 1 Name
, 
NodeList,
    Perimeter_ZN_3 Inlet Nodes,    !- Name
    Perimeter_ZN_3 Direct Air Inlet Node Name;    !- Node 1 Name
, 
NodeList,
    Perimeter_ZN_4 Inlet Nodes,    !

In [92]:
#targetidf.idfobjects['NodeList'].clear()
for i in range(len(zones)):
    nd1 = sourceidf.idfobjects['NodeList'][lst_source_AC_nodelst_idx[0]]
    targetidf.copyidfobject(nd1)
    lst_ndlst[-1].Name=nd1.Name+' '+zones[i]
    lst_ndlst[-1].Node_1_Name=nd1.Node_1_Name+' '+zones[i]   

lst_ndlst

[
NodeList,
    Core_ZN Inlet Nodes,      !- Name
    Core_ZN Direct Air Inlet Node Name;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:1_OANode List,     !- Name
    PSZ-AC:1_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:2_OANode List,     !- Name
    PSZ-AC:2_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:3_OANode List,     !- Name
    PSZ-AC:3_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:4_OANode List,     !- Name
    PSZ-AC:4_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    PSZ-AC:5_OANode List,     !- Name
    PSZ-AC:5_OAInlet Node;    !- Node 1 Name
, 
NodeList,
    Perimeter_ZN_1 Inlet Nodes,    !- Name
    Perimeter_ZN_1 Direct Air Inlet Node Name;    !- Node 1 Name
, 
NodeList,
    Perimeter_ZN_2 Inlet Nodes,    !- Name
    Perimeter_ZN_2 Direct Air Inlet Node Name;    !- Node 1 Name
, 
NodeList,
    Perimeter_ZN_3 Inlet Nodes,    !- Name
    Perimeter_ZN_3 Direct Air Inlet Node Name;    !- Node 1 Name
, 
NodeList,
    Perimeter_ZN_4 Inlet Nodes,    !

In [93]:
len(lst_ndlst)

15

In [94]:
for i in lst_target_AC_nodelst_idx:
    targetidf.idfobjects['NodeList'].pop(0)
    
len(targetidf.idfobjects['NodeList'])

5

# Edit Availability

In [95]:
#Identify HVAC related AVAILABILITY object types exist in both files. 
#AVAILABILITY needs to be capitalized as object types are capitalized in idf files.
avail_target=[s for s in target_HVAC_list if "AVAILABILITY" in s]
avail_source=[s for s in source_HVAC_list if "AVAILABILITY" in s]

#AirLoopHVAC can be different object types in target and source files. Print out the object types in sequence of target and source.
print(avail_target, avail_source)

['AVAILABILITYMANAGER:NIGHTCYCLE', 'AVAILABILITYMANAGERASSIGNMENTLIST'] ['AVAILABILITYMANAGER:SCHEDULED', 'AVAILABILITYMANAGERASSIGNMENTLIST']


Notice we have different AvailabilityManager in the target and source files. We want to keep the original AvalabilityManager in the target file to compare apple to apple. This part can be further editted based on individual's needs.

### Availability Manager

In [96]:
lst_av = targetidf.idfobjects['AvailabilityManager:NightCycle']
lst_av

[
AvailabilityManager:NightCycle,
    PSZ-AC:1 Availability Manager,    !- Name
    ALWAYS_ON,                !- Applicability Schedule Name
    HVACOperationSchd,        !- Fan Schedule Name
    CycleOnAny,               !- Control Type
    1,                        !- Thermostat Tolerance
    FixedRunTime,             !- Cycling Run Time Control Type
    1800;                     !- Cycling Run Time
, 
AvailabilityManager:NightCycle,
    PSZ-AC:2 Availability Manager,    !- Name
    ALWAYS_ON,                !- Applicability Schedule Name
    HVACOperationSchd,        !- Fan Schedule Name
    CycleOnAny,               !- Control Type
    1,                        !- Thermostat Tolerance
    FixedRunTime,             !- Cycling Run Time Control Type
    1800;                     !- Cycling Run Time
, 
AvailabilityManager:NightCycle,
    PSZ-AC:3 Availability Manager,    !- Name
    ALWAYS_ON,                !- Applicability Schedule Name
    HVACOperationSchd,        !- Fan Schedule N

In [97]:
len(lst_av)

5

### Assignment List

In [98]:
lst_avass = targetidf.idfobjects['AvailabilityManagerAssignmentList']
lst_avass

[
AvailabilityManagerAssignmentList,
    PSZ-AC:1 Availability Manager List,    !- Name
    AvailabilityManager:NightCycle,    !- Availability Manager 1 Object Type
    PSZ-AC:1 Availability Manager;    !- Availability Manager 1 Name
, 
AvailabilityManagerAssignmentList,
    PSZ-AC:2 Availability Manager List,    !- Name
    AvailabilityManager:NightCycle,    !- Availability Manager 1 Object Type
    PSZ-AC:2 Availability Manager;    !- Availability Manager 1 Name
, 
AvailabilityManagerAssignmentList,
    PSZ-AC:3 Availability Manager List,    !- Name
    AvailabilityManager:NightCycle,    !- Availability Manager 1 Object Type
    PSZ-AC:3 Availability Manager;    !- Availability Manager 1 Name
, 
AvailabilityManagerAssignmentList,
    PSZ-AC:4 Availability Manager List,    !- Name
    AvailabilityManager:NightCycle,    !- Availability Manager 1 Object Type
    PSZ-AC:4 Availability Manager;    !- Availability Manager 1 Name
, 
AvailabilityManagerAssignmentList,
    PSZ-AC:5 Availabili

In [99]:
sourceidf.idfobjects['AvailabilityManagerAssignmentList']

[
AvailabilityManagerAssignmentList,
    availability list,        !- Name
    AvailabilityManager:Scheduled,    !- Availability Manager 1 Object Type
    System availability;      !- Availability Manager 1 Name
]

Since we didn't change the AvailabilityManger, the only thing we want o modify here is the name of the assignment list to ensure it's aligned to be used in the AirLoopHVAC

In [100]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['AvailabilityManagerAssignmentList'][-1]
    lst_avass[i].Name=newclg.Name+' '+zones[i]
    lst_avass[i].Availability_Manager_1_Name=lst_av[i].Name
    
lst_avass

[
AvailabilityManagerAssignmentList,
    availability list Core_ZN,    !- Name
    AvailabilityManager:NightCycle,    !- Availability Manager 1 Object Type
    PSZ-AC:1 Availability Manager;    !- Availability Manager 1 Name
, 
AvailabilityManagerAssignmentList,
    availability list Perimeter_ZN_1,    !- Name
    AvailabilityManager:NightCycle,    !- Availability Manager 1 Object Type
    PSZ-AC:2 Availability Manager;    !- Availability Manager 1 Name
, 
AvailabilityManagerAssignmentList,
    availability list Perimeter_ZN_2,    !- Name
    AvailabilityManager:NightCycle,    !- Availability Manager 1 Object Type
    PSZ-AC:3 Availability Manager;    !- Availability Manager 1 Name
, 
AvailabilityManagerAssignmentList,
    availability list Perimeter_ZN_3,    !- Name
    AvailabilityManager:NightCycle,    !- Availability Manager 1 Object Type
    PSZ-AC:4 Availability Manager;    !- Availability Manager 1 Name
, 
AvailabilityManagerAssignmentList,
    availability list Perimeter_ZN_4, 

In [101]:
len(lst_avass)

5

# Setpoint Manager

In [102]:
#Identify HVAC related SETPOINTMANAGER object types exist in both files. 
#AVAILABILITY needs to be capitalized as object types are capitalized in idf files.
stptmgr_target=[s for s in target_HVAC_list if "SETPOINTMANAGER" in s]
stptmgr_source=[s for s in source_HVAC_list if "SETPOINTMANAGER" in s]

#AirLoopHVAC can be different object types in target and source files. Print out the object types in sequence of target and source.
print(stptmgr_target, stptmgr_source)

['SETPOINTMANAGER:SCHEDULED', 'SETPOINTMANAGER:SINGLEZONE:REHEAT', 'SETPOINTMANAGER:MIXEDAIR'] ['SETPOINTMANAGER:SCHEDULED']


Unneccessary setpoint managers have been cleared earlier, so we only need to reset the scheduled setpoint managers, which exist in both files. Check if any remained setpoint managers are HVAC related. We can reuse the function used in Branch section.

In [103]:
lst_target_AC_stptmgr_idx=find_AC_branch(keys,targetidf,'SetpointManager:Scheduled')
lst_target_AC_stptmgr_idx

[]

In [104]:
lst_source_AC_stptmgr_idx=find_AC_branch(keys,sourceidf,'SetpointManager:Scheduled')
lst_source_AC_stptmgr_idx

[]

Both files have no AC related SetpointManager:Scheduled. No actions needed.

# AirLoopHVAC

Now with all the components setup and checked, we can finally come back to the object type AirLoopHVAC. Here we connect all the previous pieces together following the logic mentioned earlier.

In [105]:
lst_ALHVAC = targetidf.idfobjects['AirLoopHVAC']
lst_ALHVAC

[
AirLoopHVAC,
    PSZ-AC:1,                 !- Name
    ,                         !- Controller List Name
    PSZ-AC:1 Availability Manager List,    !- Availability Manager List Name
    AUTOSIZE,                 !- Design Supply Air Flow Rate
    PSZ-AC:1 Air Loop Branches,    !- Branch List Name
    ,                         !- Connector List Name
    PSZ-AC:1 Supply Equipment Inlet Node,    !- Supply Side Inlet Node Name
    PSZ-AC:1 Zone Equipment Outlet Node,    !- Demand Side Outlet Node Name
    PSZ-AC:1 Zone Equipment Inlet Node,    !- Demand Side Inlet Node Names
    PSZ-AC:1 Supply Equipment Outlet Node;    !- Supply Side Outlet Node Names
, 
AirLoopHVAC,
    PSZ-AC:2,                 !- Name
    ,                         !- Controller List Name
    PSZ-AC:2 Availability Manager List,    !- Availability Manager List Name
    AUTOSIZE,                 !- Design Supply Air Flow Rate
    PSZ-AC:2 Air Loop Branches,    !- Branch List Name
    ,                         !- Connect

In [106]:
sourceidf.idfobjects['AirLoopHVAC']

[
AirLoopHVAC,
    Central System_unit1,     !- Name
    ,                         !- Controller List Name
    availability list,        !- Availability Manager List Name
    autosize,                 !- Design Supply Air Flow Rate
    Air Loop Branches_unit1,    !- Branch List Name
    ,                         !- Connector List Name
    Air Loop Inlet Node_unit1,    !- Supply Side Inlet Node Name
    Return Air Mixer Outlet_unit1,    !- Demand Side Outlet Node Name
    Zone Equipment Inlet Node_unit1,    !- Demand Side Inlet Node Names
    Air Loop Outlet Node_unit1;    !- Supply Side Outlet Node Names
]

In [107]:
len(lst_branchlst)-len(zones)

2

In [108]:
targetidf.idfobjects['AirLoopHVAC'].clear()
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['AirLoopHVAC'][-1]
    targetidf.copyidfobject(newclg)
    lst_ALHVAC[-1].Name=newclg.Name+' '+zones[i]
    lst_ALHVAC[-1].Availability_Manager_List_Name=lst_avass[i].Name #From AvailabilityManagerAssignmentList
    lst_ALHVAC[-1].Branch_List_Name=lst_branchlst[2+i].Name #From BranchList
    lst_ALHVAC[-1].Supply_Side_Inlet_Node_Name=newclg.Supply_Side_Inlet_Node_Name+' '+zones[i]
    lst_ALHVAC[-1].Demand_Side_Outlet_Node_Name=newclg.Demand_Side_Outlet_Node_Name+' '+zones[i]
    lst_ALHVAC[-1].Demand_Side_Inlet_Node_Names=newclg.Demand_Side_Inlet_Node_Names+' '+zones[i]
    lst_ALHVAC[-1].Supply_Side_Outlet_Node_Names=newclg.Supply_Side_Outlet_Node_Names+' '+zones[i]

lst_ALHVAC

[
AirLoopHVAC,
    Central System_unit1 Core_ZN,    !- Name
    ,                         !- Controller List Name
    availability list Core_ZN,    !- Availability Manager List Name
    autosize,                 !- Design Supply Air Flow Rate
    Air Loop Branches_unit1 Core_ZN,    !- Branch List Name
    ,                         !- Connector List Name
    Air Loop Inlet Node_unit1 Core_ZN,    !- Supply Side Inlet Node Name
    Return Air Mixer Outlet_unit1 Core_ZN,    !- Demand Side Outlet Node Name
    Zone Equipment Inlet Node_unit1 Core_ZN,    !- Demand Side Inlet Node Names
    Air Loop Outlet Node_unit1 Core_ZN;    !- Supply Side Outlet Node Names
, 
AirLoopHVAC,
    Central System_unit1 Perimeter_ZN_1,    !- Name
    ,                         !- Controller List Name
    availability list Perimeter_ZN_1,    !- Availability Manager List Name
    autosize,                 !- Design Supply Air Flow Rate
    Air Loop Branches_unit1 Perimeter_ZN_1,    !- Branch List Name
    ,       

# Curves

Curves and Schedules are different from the previous object types. We want to make sure all the new curves and schedules are migrated properly to avoid any errors, but also want to avoid duplicated names in Curves and Schedules. For curves, the safest way to do this is to duplicate all the Curves in the source file and then check any duplicated names exist. We will then drop the object with the duplicated name in the TARGET file - the first object that show up. Below are the functions to do so.

In [109]:
def find_dup_and_identify_index(target_lst):
    
    # Dictionary to track the indices of duplicates
    index_dict = {}
    duplicates = []
    
    # Identify indices of duplicated items
    for idx, item in enumerate(target_lst):
        if item in index_dict:
            duplicates.append(index_dict[item])  # Record the first occurrence index
            index_dict[item] = idx  # Update to the latest index
        else:
            index_dict[item] = idx
    
    return duplicates

In [110]:
def drop_target_duplication(index_lst, idf, obj_type):
    
    #If the length of duplication index list is not zero, pop the curve objet, else do nothing
    if len(index_lst) > 0:
        
        #VERY IMPORTANT: Iterate backwards over the indices to avoid shifting issues
        for idx in reversed(index_lst):
            idf.idfobjects[obj_type].pop(idx)
        

In [111]:
#Identify HVAC related CURVE object types exist in both files. 
#AVAILABILITY needs to be capitalized as object types are capitalized in idf files.
curve_target=[s for s in target_HVAC_list if "CURVE" in s]
curve_source=[s for s in source_HVAC_list if "CURVE" in s]

#AirLoopHVAC can be different object types in target and source files. Print out the object types in sequence of target and source.
print(curve_target, curve_source)

['CURVE:QUADRATIC', 'CURVE:CUBIC', 'CURVE:BIQUADRATIC'] ['CURVE:QUADRATIC', 'CURVE:CUBIC', 'CURVE:BIQUADRATIC']


This means that no new type of curve is involved in the source file. This can make the curve replacement and addition process much easier. We can handle all types of Curve all together.

In [112]:
if curve_target == curve_source:
    
    for curve_type in curve_source:
        lst_curve = targetidf.idfobjects[curve_type]
        new_curves = sourceidf.idfobjects[curve_type][:] #We will migrate all curves from the source file
        
        for i in range(len(new_curves)):
            targetidf.copyidfobject(new_curves[i])
        
        #Now we start the duplication check
        lst_curve_name=[]
        for i in range(len(lst_curve)):
            lst_curve_name.append(lst_curve[i].Name)
        dup_curve_index=find_dup_and_identify_index(lst_curve_name)
        
        #If any duplicated curve names exist, drop the first one that existed in the original target file
        drop_target_duplication(dup_curve_index, targetidf, curve_type)

In case the target and source files have different Curve types, we will simply repeat the above process but adding new curve types from the source file to the target file.

In [113]:
if curve_target != curve_source:
    
    for i in range(len(curve_source)):
        
        #When there are new curve types in the source file
        if curve_source[i] not in curve_target:
            lst_curve = targetidf.idfobjects[curve_source[i]] #Supposed to be an empty list
            new_curves = sourceidf.idfobjects[curve_source[i]][:]
            for a in range(len(new_curves)):
                targetidf.copyidfobject(new_curves[a]) #No need to check duplications
        
        #Then repeat the same process for those same existing curve types
        elif curve_source[i] in curve_target:
            lst_curve = targetidf.idfobjects[curve_source[i]] #Supposed to be a non-empty list
            new_curves = sourceidf.idfobjects[curve_source[i]][:]
            for b in range(len(new_curves)):
                targetidf.copyidfobject(new_curves[b]) #Need to check duplications
            #Check for duplications
            lst_curve_name=[]
            for c in range(len(lst_curve)):
                lst_curve_name.append(lst_curve[c].Name)
            dup_curve_index=find_dup_and_identify_index(lst_curve_name)
            #If any duplicated curve names exist, drop the first one that existed in the original target file
            drop_target_duplication(dup_curve_index, targetidf, curve_source[i])

    

# Schedule

The remaining item is to check if any shedules should be added from source to target. We first review all used object types with all associated fieldnames in the target file.

In [114]:
#Create two empty lists, one for object type, one for the associated fieldnames
obj_types=[]
obj_type_fieldnames=[]

#Pull out all used object type and associated fieldnames
for obj_type in source_HVAC_list:
    if len(targetidf.idfobjects[obj_type]) > 0:
        obj=targetidf.idfobjects[obj_type][0]
        obj_types.append(obj_type)
        fieldnames=obj.fieldnames
        obj_type_fieldnames.append(fieldnames)

#Organize the object types and fieldnames into a dictionary
obj_fieldnames_dict=dict(zip(obj_types, obj_type_fieldnames))
obj_fieldnames_dict

{'SIZING:SYSTEM': ['key',
  'AirLoop_Name',
  'Type_of_Load_to_Size_On',
  'Design_Outdoor_Air_Flow_Rate',
  'Central_Heating_Maximum_System_Air_Flow_Ratio',
  'Preheat_Design_Temperature',
  'Preheat_Design_Humidity_Ratio',
  'Precool_Design_Temperature',
  'Precool_Design_Humidity_Ratio',
  'Central_Cooling_Design_Supply_Air_Temperature',
  'Central_Heating_Design_Supply_Air_Temperature',
  'Type_of_Zone_Sum_to_Use',
  '100_Outdoor_Air_in_Cooling',
  '100_Outdoor_Air_in_Heating',
  'Central_Cooling_Design_Supply_Air_Humidity_Ratio',
  'Central_Heating_Design_Supply_Air_Humidity_Ratio',
  'Cooling_Supply_Air_Flow_Rate_Method',
  'Cooling_Supply_Air_Flow_Rate',
  'Cooling_Supply_Air_Flow_Rate_Per_Floor_Area',
  'Cooling_Fraction_of_Autosized_Cooling_Supply_Air_Flow_Rate',
  'Cooling_Supply_Air_Flow_Rate_Per_Unit_Cooling_Capacity',
  'Heating_Supply_Air_Flow_Rate_Method',
  'Heating_Supply_Air_Flow_Rate',
  'Heating_Supply_Air_Flow_Rate_Per_Floor_Area',
  'Heating_Fraction_of_Autosized_

Now we narrow down the dictionary to those fieldnames containing "Schedule". We only keep the object type and its fieldnames that have "Schedule".

In [115]:
schedule_key='Schedule'
obj_fieldnames_dict_with_schedule = {
    k: [value for value in v if schedule_key in value]
    for k, v in obj_fieldnames_dict.items() 
    if any(schedule_key in value for value in v)
}
obj_fieldnames_dict_with_schedule

{'AIRTERMINAL:SINGLEDUCT:CONSTANTVOLUME:NOREHEAT': ['Availability_Schedule_Name'],
 'ZONEHVAC:EQUIPMENTLIST': ['Zone_Equipment_1_Sequential_Cooling_Fraction_Schedule_Name',
  'Zone_Equipment_1_Sequential_Heating_Fraction_Schedule_Name',
  'Zone_Equipment_2_Sequential_Cooling_Fraction_Schedule_Name',
  'Zone_Equipment_2_Sequential_Heating_Fraction_Schedule_Name',
  'Zone_Equipment_3_Sequential_Cooling_Fraction_Schedule_Name',
  'Zone_Equipment_3_Sequential_Heating_Fraction_Schedule_Name',
  'Zone_Equipment_4_Sequential_Cooling_Fraction_Schedule_Name',
  'Zone_Equipment_4_Sequential_Heating_Fraction_Schedule_Name',
  'Zone_Equipment_5_Sequential_Cooling_Fraction_Schedule_Name',
  'Zone_Equipment_5_Sequential_Heating_Fraction_Schedule_Name',
  'Zone_Equipment_6_Sequential_Cooling_Fraction_Schedule_Name',
  'Zone_Equipment_6_Sequential_Heating_Fraction_Schedule_Name',
  'Zone_Equipment_7_Sequential_Cooling_Fraction_Schedule_Name',
  'Zone_Equipment_7_Sequential_Heating_Fraction_Schedule_Na

Dissecting the dictionary, we have dictionary keys indicating object types that use schedule as input.

In [116]:
obj_type_with_schedule=list(obj_fieldnames_dict_with_schedule.keys())
obj_type_with_schedule

['AIRTERMINAL:SINGLEDUCT:CONSTANTVOLUME:NOREHEAT',
 'ZONEHVAC:EQUIPMENTLIST',
 'ZONEHVAC:EQUIPMENTCONNECTIONS',
 'FAN:ONOFF',
 'COIL:COOLING:DX:SINGLESPEED',
 'COIL:HEATING:ELECTRIC',
 'COIL:HEATING:DX:SINGLESPEED',
 'AIRLOOPHVAC:UNITARYHEATPUMP:AIRTOAIR',
 'SETPOINTMANAGER:SCHEDULED']

Now we can pull out all schedules used by the new HVAC system.

In [117]:
used_schedules = []
for obj_type in obj_type_with_schedule:
    
    #Pull out the list of fieldnames with "Schedule"
    schedule_field = obj_fieldnames_dict_with_schedule[obj_type]
    
    #Check each field and pull out the name of the schedule used
    for field in schedule_field:
        schedule = getattr(targetidf.idfobjects[obj_type][0], field, None)
        if schedule != None:
            used_schedules.append(schedule)

#Remove empty schedules
used_schedules = [sch for sch in used_schedules if sch != '']

In [118]:
used_schedules

['always_avail',
 'always_avail',
 'always_avail',
 'always_avail',
 'always_avail',
 'always_avail',
 'SWHSys1-Loop-Temp-Schedule']

Now we go back to the existing schedules. There are a few different object types for schedules.

In [119]:
schedule_obj_type = [sch for sch in unique_target_list if 'SCHEDULE' in sch]
schedule_obj_type

['SCHEDULETYPELIMITS', 'SCHEDULE:COMPACT', 'SETPOINTMANAGER:SCHEDULED']

We review each schedule type and pull out the name of all existing schedules.

In [120]:
existing_schedules=[]
for sch_type in schedule_obj_type:
    for i in range(len(targetidf.idfobjects[sch_type])):
        sch = targetidf.idfobjects[sch_type][i].Name
        existing_schedules.append(sch)

existing_schedules

['Any Number',
 'Fraction',
 'Temperature',
 'On/Off',
 'Control Type',
 'Humidity',
 'Number',
 'ALWAYS_ON',
 'INFIL_QUARTER_ON_SCH',
 'BLDG_OCC_SCH',
 'BLDG_LIGHT_SCH',
 'BLDG_EQUIP_SCH',
 'ACTIVITY_SCH',
 'WORK_EFF_SCH',
 'AIR_VELO_SCH',
 'CLOTHING_SCH',
 'CLGSETP_SCH',
 'HTGSETP_SCH',
 'MinOA_Sched',
 'Dual Zone Control Type Sched',
 'HVACOperationSchd',
 'BLDG_SWH_SCH',
 'Water Equipment Latent fract sched',
 'Water Equipment Sensible fract sched',
 'SWHSys1 Water Heater Ambient Temperature Schedule Name',
 'Water Equipment Temp Sched',
 'Water Equipment Hot Supply Temp Sched',
 'SWHSys1 Water Heater Setpoint Temperature Schedule Name',
 'SWHSys1-Loop-Temp-Schedule',
 'ReliantEnergySeasonSched',
 'SWHSys1 Loop Setpoint Manager']

Check if any used schedules are not prepared in the Schedule objects. Luckily we don't!

In [121]:
new_sch_needed = list(set(used_schedules) - set(existing_schedules))
new_sch_needed

['always_avail']

If new schedules need to be added, we proceed with the following steps.

In [122]:
matching_objects = []

if len(new_sch_needed) > 0:    
    for sch in new_sch_needed:
    
    # Iterate through all objects in the source file
        for obj_type in sourceidf.idfobjects:  # Loop through all object types
            for obj in sourceidf.idfobjects[obj_type]:  # Loop through all objects of that type
                
                # Check if the schedule name appears in any field of the object
                if sch in obj.fieldvalues:
                    if "SCHEDULE" in obj_type: #Make sure only the SCHEDULE object is included, not the object that uses the schedule
                        matching_objects.append(obj)

    
    #Copy the new schedules to the target file
    for new_obj in matching_objects:
        targetidf.copyidfobject(new_obj)



# Remove AirFlowNetwork

DOE has released AirFlowNetwork modules embedded only in prototype SFH models. As a parallel modeling secotr, it does not interfer the processing of energy modeling, but its existence can lead to some unexpected errors with the replacement of the new HVAC system - adding or removing certain components can lead to fatal errors in AirFlowNetwork. For convenience, we remove the AirFlowNetwork objects embedded in the target file to avoid unnecessary errors.

In [123]:
#For this file only: Remove all AirFlowNetwork objects
for obj_type in targetidf.idfobjects:
    if 'AIRFLOWNETWORK' in obj_type:
        targetidf.idfobjects[obj_type].clear()

# Save as

There we are! Save the modified target file to a new file - HVAC model electrified!

In [124]:
targetidf.saveas('RefBldgSmallOfficePost1980_v1.4_7.2_2A_USA_TX_HOUSTON_ASHP_EnergyStar.idf')

TBD Question to be answered in Lit Review: Why Not OpenStudio Paramtric System?

Benefits: can be applied universally regardless baseline conditions - target file can be different building types, different exisitng systems, different climate zones. Minimal amount of manual work is involved, faster to proceed than interface like OpenStudio.

Inflexible part: something like AirflowNetwork, ERV, etc., if not existed in the target file, better to prepare it in the source file first, then deployed to target file.